# Comparison of fibroblast populations

In this notebook we are going to extract and replicate the main populations from diffrent papers where fibroblast populations are described, and find similarities and differences. The premise of this analysis is that many of the populations described in different papers seem not to match, or to be transcriptomically different, but in reality they are quite similar; that is, the main types of populations are indeed shared by the different papers, which should come as no surprise.

**After the publication in JID we will include the following papers, as confirmatory results**
* Tabib 2018
* Solé-Boldo 2020
* Vorstandlechner 2020
* He 2020
* Kim 2020
* Gaydosik 2020
* McCarthy 2020
* Mirizio 2020
* Gao 2021
* Reynolds 2021

Additionally, we will reanalize the *classic 4* papers, to check that cell populations are assigned as expected. For these papers, UMAPs might vary compared to the ones in our paper, but the main results should still be the same.

## imports

In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import scipy.sparse as spr
# import ray
# import subprocess
# import time
# import scvelo as scv
# import gc
# import gseapy as gp

In [ ]:
!pip install cellassign

In [ ]:
from cellassign import assign_cats

In [ ]:
# To print versions of imports 

import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    try:
        name, version = module.split('==')
        if name in changed_imported_modules:
            print(name + '\t' + version)
    except:
        pass

In [ ]:
seed = 0

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
dict_cats_fb = {'peri': ['Rgs5', 'Myl9', 'Ndufa4l2', 'Nrip2', 'Mylk', 'Rgs4', 'Acta2', 'Sncg', 'Tagln', 'Des', 'Ptp4a3', 'Myh11'], 
                'endo': ['Pecam1', 'Cdh5', 'Egfl7', 'Cd36', 'Srgn', 'Adgrf5', 'Ptprb', 'Scarb1', 'Plvap', 'Grrp1', 'C1qtnf9', 'Mmrn2', 'Flt1'], 
                'kerato': ['Krt14', 'Krt15', 'Perp', 'S100a14', 'Ccl27a', 'Gata3', 'Dapl1', 'Rab25', 'Ckmt1', 'Col17a1', 'Serpinb5'],
                'kerato Gjb2': ['Ucp2', 'Krt71', 'Gjb2', 'Ahcy', 'Acaa2', 'Cbs', 'Slc3a2', 'Serpina11', 'Lap3', 'Gss', 'Basp1', ],
                'fibro': ['Dcn', 'Pdgfra',  'Lum', 'Col1a1', 'Col1a2',],
                'fibro_2': ['Ncam1', 'Ptch1', 'Trps1', 'Col11a1', 'Wif1'],
                'T cell': ['Rac2', 'Ptprcap', 'Il2rg', 'Cd3g', 'Skap1', 'Hcst', 'Ctsw', 'Ets1', 'Cd3d', 'Ctla2a', 'Cd2'],
                'APC': ['Tyrobp', 'Cd74', 'H2-Aa', 'H2-Eb1', 'Ctss', 'Spi1', 'Napsa', 'Cd68', 'Lyz2', 'Csf2ra'],
                'lymph': ['Ccl21a', 'Egfl7', 'Mmrn1', 'Nsg1', 'Meox1', 'Gimap6', 'Kdr'],
                'melano / schwann': ['Syngr1', 'Pmel', 'Mlana'],
                'myo': ['Tnnt1', 'Tnnt2', 'Tnnt3', 'Tnnc2', 'Acta1', 'Myl1', 'Tnni2', 'Tcap', 'Eno3', 'Myoz1'],
                'neural': ['Itgb8', 'Plp1', 'Ptn', 'Egfl8', 'Chl1', 'Cadm4', 'Sox10', 'Cdh19', 'Snca']
               }

provisional_manual_dict_cats_axes = {}
provisional_manual_dict_cats_clusters = {}

In [ ]:
dict_colors = {'a1': '#c93038', 'a2': '#de6a38', 'a3': '#ffad3b', 'a4': '#852d66',
               'b1': '#b4d645', 'b2': '#51c43f', 'b3': '#309c63',
               'c1': '#93dfe4', 'c2': '#63c2c9', 'c3': '#4c93ad',  'c4': '#3c83bd', 'c5': '#264f6e', 'c6': '#1c735d',
               'd1': '#fcbf8a', 'd2': '#b58057', 'd3': '#956642',  'd4': '#754622',
               'T1': '#29c2a8', 'U': '#dedede'}

In [ ]:
dict_rep = {'CCN5': 'WISP2', 'ECRG4': 'C2orf40'}

In [ ]:
mpl.rcParams['figure.dpi'] = 150

In [ ]:
def plot_score_graph(adatax):
    df_cats_own = pd.DataFrame(index=adatax.obs_names, columns=['clusters', 'score'])
    for cluster in adatax.obs['cluster'].cat.categories:
        adata_sub = adatax[adatax.obs['cluster'] == cluster]
        try:
            df_cats_own.loc[adata_sub.obs_names, 'score'] = adata_sub.obs[f'cluster_{cluster}']
            df_cats_own.loc[adata_sub.obs_names, 'clusters'] = cluster
        except:
            pass

    df_cats_own = df_cats_own.sort_values('clusters')
    sns.barplot(x='clusters', y='score', data=df_cats_own, palette=adatax.uns['cluster_colors'])

In [ ]:
data_dir = os.getcwd() + '/data/'
print(data_dir)

**IMPORTANT: I am running this analysis in a computer with ~500 GB of RAM. I will load many datasets at once, which might be too much for some computers. I took this decision conciously, to have as much info available at any time as possible. If you cannot run all the analysis at once, you can run it by parts.**

## Abbasi et al. 2020

In [ ]:
abbasi_2020_dir = data_dir + '/abassi_2020'

In [ ]:
abassi_2020_ctrl = sc.read(f"{abbasi_2020_dir}/abassi_2020_ctrl.h5")

In [ ]:
# Basic QC filtering
abassi_2020_ctrl.var['mt'] = abassi_2020_ctrl.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(abassi_2020_ctrl, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(abassi_2020_ctrl, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(abassi_2020_ctrl, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(abassi_2020_ctrl, x='log1p_total_counts', y='log1p_n_genes_by_counts')

In [ ]:
abassi_2020_ctrl = abassi_2020_ctrl[(
                              ((abassi_2020_ctrl.obs.n_genes_by_counts < 3000) & 
                                (abassi_2020_ctrl.obs.n_genes_by_counts > 1100))).values, :]
abassi_2020_ctrl = abassi_2020_ctrl[abassi_2020_ctrl.obs.pct_counts_mt < 9, :]

In [ ]:
sc.pp.filter_genes(abassi_2020_ctrl, min_counts=1)
sc.pp.normalize_total(abassi_2020_ctrl)
sc.pp.log1p(abassi_2020_ctrl)

In [ ]:
sc.pp.pca(abassi_2020_ctrl, random_state=seed, n_comps=50)
sc.pp.neighbors(abassi_2020_ctrl, n_neighbors=int(0.5 * len(abassi_2020_ctrl) ** 0.5), metric='cosine')
tk.tl.triku(abassi_2020_ctrl)

In [ ]:
sc.tl.umap(abassi_2020_ctrl, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(abassi_2020_ctrl, resolution=2, random_state=seed)

In [ ]:
assign_cats(abassi_2020_ctrl, dict_cats=dict_cats_fb, min_score=0.5)
sc.pl.umap(abassi_2020_ctrl, color=['leiden', 'assigned_cats', 'Pdgfra', 'Lum', 'Col1a1', 'Coch', 'Vim'], legend_loc='on data', cmap=magma, ncols=2, 
           use_raw=False, )

In [ ]:
abassi_2020_ctrl_fb = abassi_2020_ctrl[abassi_2020_ctrl.obs['assigned_cats'].isin(['fibro', 'fibro_2'])]

In [ ]:
sc.pp.filter_genes(abassi_2020_ctrl_fb, min_counts=1)

In [ ]:
sc.pp.pca(abassi_2020_ctrl_fb, random_state=seed, n_comps=25)
sc.pp.neighbors(abassi_2020_ctrl_fb, n_neighbors=int(0.5 * len(abassi_2020_ctrl_fb) ** 0.5), metric='cosine')
tk.tl.triku(abassi_2020_ctrl_fb)

In [ ]:
sc.tl.umap(abassi_2020_ctrl_fb, random_state=seed)

In [ ]:
sc.tl.leiden(abassi_2020_ctrl_fb, resolution=5, random_state=seed)

In [ ]:
sc.pl.umap(abassi_2020_ctrl_fb, color=['leiden'], 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

In [ ]:
sc.tl.rank_genes_groups(abassi_2020_ctrl_fb, groupby='leiden', method='wilcoxon')

In [ ]:
sc.pl.umap(abassi_2020_ctrl_fb, color=['leiden'] + ['Ccl19', 'Ndufa4l2', 'Mfap4', 'Eln', 'Tnmd', 'Cyp2f2', 'Tsc22d3', 'Rtn4r', 'Ltbp2', 'Gpm6b', 'Fam180a', 'Lsamp', 'Pid1', 'Lgr5', 'Sectm1a', 'Prkcb', 'Entpd1',  'Serpina3c', 'Myo1b', 'H2-Q7', 'Bmp4', 'Cystm1', 'Cyp1b1', 'Wnt5b', 'Id1', 'Syt13', 'Etv1', 'Eya1'], 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

## Buechler et al. 2021

In [ ]:
buechler_2021_dir = data_dir + '/buechler_2021'

In [ ]:
buechler_2021_ctrl = sc.read(buechler_2021_dir + '/buechler_2021_ctrl.h5')
buechler_2021_ctrl.var_names_make_unique()

In [ ]:
# Basic QC filtering
buechler_2021_ctrl.var['mt'] = buechler_2021_ctrl.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(buechler_2021_ctrl, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(buechler_2021_ctrl, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(buechler_2021_ctrl, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(buechler_2021_ctrl, x='total_counts', y='n_genes_by_counts')

In [ ]:
buechler_2021_ctrl = buechler_2021_ctrl[((buechler_2021_ctrl.obs.n_genes_by_counts < 4000) & 
                                    (buechler_2021_ctrl.obs.n_genes_by_counts > 2000)).values, :]
buechler_2021_ctrl = buechler_2021_ctrl[buechler_2021_ctrl.obs.pct_counts_mt < 12, :]

In [ ]:
sc.pp.filter_genes(buechler_2021_ctrl, min_counts=1)
sc.pp.normalize_total(buechler_2021_ctrl)
sc.pp.log1p(buechler_2021_ctrl)

In [ ]:
sc.pp.pca(buechler_2021_ctrl, random_state=seed, n_comps=50)
sc.pp.neighbors(buechler_2021_ctrl, n_neighbors=int(0.5 * len(buechler_2021_ctrl) ** 0.5 // 2), metric='cosine')
tk.tl.triku(buechler_2021_ctrl)

In [ ]:
sc.tl.umap(buechler_2021_ctrl, min_dist=0.2, random_state=seed)
sc.tl.leiden(buechler_2021_ctrl, resolution=4, random_state=seed)

In [ ]:
sc.pp.subsample(buechler_2021_ctrl, fraction=1, random_state=0, copy=False)
sc.pl.umap(buechler_2021_ctrl, color=['leiden'], legend_loc='on data')

In [ ]:
assign_cats(buechler_2021_ctrl, dict_cats=dict_cats_fb, min_score=0.35, quantile_gene_sel=0.85)

In [ ]:
sc.pl.umap(buechler_2021_ctrl, color=['leiden', 'assigned_cats'], legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
buechler_2021_ctrl_fb = buechler_2021_ctrl[buechler_2021_ctrl.obs['assigned_cats'].isin(['fibro', 'fibro_2'])]

In [ ]:
sc.pp.filter_genes(buechler_2021_ctrl_fb, min_counts=1)

In [ ]:
sc.pp.pca(buechler_2021_ctrl_fb, random_state=seed, n_comps=30)
sc.pp.neighbors(buechler_2021_ctrl_fb,  n_neighbors=int(len(buechler_2021_ctrl_fb) ** 0.5 // 6), metric='cosine')
tk.tl.triku(buechler_2021_ctrl_fb)

In [ ]:
sc.tl.umap(buechler_2021_ctrl_fb, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(buechler_2021_ctrl_fb, resolution=10, random_state=seed)
# sc.tl.leiden(buechler_2021_ctrl_fb, resolution=0.2, random_state=seed)

In [ ]:
sc.pl.umap(buechler_2021_ctrl_fb, color=['leiden', 'Itga6', 'Lamc3'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

## Efremova, Mirjana 2018

In [ ]:
efremova_2018_dir = data_dir + '/efremova_2018'

In [ ]:
efremova_2018_ctrl = sc.read(f"{efremova_2018_dir}/efremova_2018_ctrl.h5")

In [ ]:
# Basic QC filtering
efremova_2018_ctrl.var['mt'] = efremova_2018_ctrl.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(efremova_2018_ctrl, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(efremova_2018_ctrl, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(efremova_2018_ctrl, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(efremova_2018_ctrl, x='total_counts', y='n_genes_by_counts')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': efremova_2018_ctrl.obs['Internal sample identifier'], 'y': efremova_2018_ctrl.obs['log1p_n_genes_by_counts']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': efremova_2018_ctrl.obs['Internal sample identifier'], 'y': efremova_2018_ctrl.obs['pct_counts_mt']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
efremova_2018_ctrl = efremova_2018_ctrl[(
                              ((efremova_2018_ctrl.obs['Internal sample identifier'] == '0') & (efremova_2018_ctrl.obs.log1p_n_genes_by_counts < 8.3) & 
                                (efremova_2018_ctrl.obs.log1p_n_genes_by_counts > 6.5))).values, :]
efremova_2018_ctrl = efremova_2018_ctrl[efremova_2018_ctrl.obs.pct_counts_mt < 9, :]

In [ ]:
sc.pp.filter_genes(efremova_2018_ctrl, min_counts=1)
sc.pp.normalize_total(efremova_2018_ctrl)
sc.pp.log1p(efremova_2018_ctrl)

In [ ]:
sc.pp.pca(efremova_2018_ctrl, random_state=seed, n_comps=50)
sc.pp.neighbors(efremova_2018_ctrl, n_neighbors=int(0.5 * len(efremova_2018_ctrl) ** 0.5), metric='cosine')
tk.tl.triku(efremova_2018_ctrl)

In [ ]:
sc.tl.umap(efremova_2018_ctrl, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(efremova_2018_ctrl, resolution=2, random_state=seed)

In [ ]:
assign_cats(efremova_2018_ctrl, dict_cats=dict_cats_fb, min_score=0.5)
sc.pl.umap(efremova_2018_ctrl, color=['leiden', 'Internal sample identifier', 'assigned_cats', 'Pdgfra', 'Lum', 'Col1a1', 'Coch', 'Vim'], legend_loc='on data', cmap=magma, ncols=2, 
           use_raw=False, )

In [ ]:
efremova_2018_ctrl_fb = efremova_2018_ctrl[efremova_2018_ctrl.obs['assigned_cats'].isin(['fibro', 'fibro_2'])]

In [ ]:
sc.pp.filter_genes(efremova_2018_ctrl_fb, min_counts=1)

In [ ]:
sc.pp.pca(efremova_2018_ctrl_fb, random_state=seed, n_comps=25)
sc.pp.neighbors(efremova_2018_ctrl_fb, n_neighbors=int(0.5 * len(efremova_2018_ctrl_fb) ** 0.5), metric='cosine')
tk.tl.triku(efremova_2018_ctrl_fb)

In [ ]:
sc.tl.umap(efremova_2018_ctrl_fb, random_state=seed)

In [ ]:
sc.tl.leiden(efremova_2018_ctrl_fb, resolution=0.6, random_state=seed)

In [ ]:
sc.pl.umap(efremova_2018_ctrl_fb, color=['leiden', 'Internal sample identifier'], 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

In [ ]:
sc.tl.rank_genes_groups(efremova_2018_ctrl_fb, groupby='leiden', method='wilcoxon')

In [ ]:
sc.pl.umap(efremova_2018_ctrl_fb, color=['leiden'] + list(efremova_2018_ctrl_fb.uns['rank_genes_groups']['names']['0'][:250]), 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
sc.pl.umap(efremova_2018_ctrl_fb, color=['leiden'] + list(efremova_2018_ctrl_fb.uns['rank_genes_groups']['names']['2'][:250]), 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
sc.pl.umap(efremova_2018_ctrl_fb, color=['leiden'] + list(efremova_2018_ctrl_fb.uns['rank_genes_groups']['names']['3'][:250]), 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
sc.pl.umap(efremova_2018_ctrl_fb, color=['leiden'] + list(efremova_2018_ctrl_fb.uns['rank_genes_groups']['names']['1'][:250]), 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
sc.pl.umap(efremova_2018_ctrl_fb, color=['leiden'] + list(efremova_2018_ctrl_fb.uns['rank_genes_groups']['names']['4'][:250]), 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
sc.pl.umap(efremova_2018_ctrl_fb, color=['leiden'] + list(efremova_2018_ctrl_fb.uns['rank_genes_groups']['names']['5'][:250]), 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
efremova_2018_ctrl

## Haensel et al. 2021

**CAUTION** We include **wounded and unwounded** samples! We have seen that B1 population (B general population) from this dataset was almost depleted in the unwounded, and included with more heterogeneity in the wounded state. However, these populations appear in other datasets such as Abassi and Shook (at homeostatic states), so we are going to keep them.

In [ ]:
haensel_2021_dir = data_dir + '/haensel_2021'

In [ ]:
haensel_2021_ctrl = sc.read(f"{haensel_2021_dir}/haensel_2021_ctrl.h5")
haensel_2021_wounding = sc.read(f"{haensel_2021_dir}/haensel_2021_wounding.h5")

In [ ]:
haensel_2021_ctrl_wounding = sc.AnnData.concatenate(haensel_2021_ctrl, haensel_2021_wounding, batch_categories=['Healthy', 'Wounding'],
                                           batch_key='Condition')


In [ ]:
# Basic QC filtering
haensel_2021_ctrl_wounding.var['mt'] = haensel_2021_ctrl_wounding.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(haensel_2021_ctrl_wounding, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(haensel_2021_ctrl_wounding, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(haensel_2021_ctrl_wounding, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(haensel_2021_ctrl_wounding, x='total_counts', y='n_genes_by_counts')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': haensel_2021_ctrl_wounding.obs['Internal sample identifier'], 'y': haensel_2021_ctrl_wounding.obs['log1p_n_genes_by_counts']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': haensel_2021_ctrl_wounding.obs['Internal sample identifier'], 'y': haensel_2021_ctrl_wounding.obs['pct_counts_mt']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
haensel_2021_ctrl_wounding = haensel_2021_ctrl_wounding[(((haensel_2021_ctrl_wounding.obs.pct_counts_mt < 10) &
                                (haensel_2021_ctrl_wounding.obs.log1p_n_genes_by_counts > 7.2) & 
                                (haensel_2021_ctrl_wounding.obs.log1p_n_genes_by_counts < 8.1)) 
                                            ).values, :]

In [ ]:
sc.pp.filter_genes(haensel_2021_ctrl_wounding, min_counts=1)
sc.pp.normalize_total(haensel_2021_ctrl_wounding)
sc.pp.log1p(haensel_2021_ctrl_wounding)

In [ ]:
sc.pp.pca(haensel_2021_ctrl_wounding, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(haensel_2021_ctrl_wounding, key='Internal sample identifier', max_iter_harmony=50)
sc.pp.neighbors(haensel_2021_ctrl_wounding, use_rep='X_pca_harmony', n_neighbors=int(0.5 * len(haensel_2021_ctrl_wounding) ** 0.5 // 4), metric='cosine')
tk.tl.triku(haensel_2021_ctrl_wounding)

In [ ]:
sc.tl.umap(haensel_2021_ctrl_wounding, min_dist=0.2, random_state=seed)

In [ ]:
sc.tl.leiden(haensel_2021_ctrl_wounding, resolution=0.2, random_state=seed)

In [ ]:
assign_cats(haensel_2021_ctrl_wounding, dict_cats=dict_cats_fb, min_score=0.4, quantile_gene_sel=0.4)
sc.pl.umap(haensel_2021_ctrl_wounding, color=['leiden', 'assigned_cats', 'Internal sample identifier'], legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
haensel_2021_ctrl_wounding_fb = haensel_2021_ctrl_wounding[haensel_2021_ctrl_wounding.obs['assigned_cats'].isin(['fibro', 'fibro_2'])]

In [ ]:
sc.pp.filter_genes(haensel_2021_ctrl_wounding_fb, min_counts=1)

In [ ]:
sc.pp.pca(haensel_2021_ctrl_wounding_fb, random_state=seed, n_comps=35)
sce.pp.harmony_integrate(haensel_2021_ctrl_wounding_fb, key='Internal sample identifier', max_iter_harmony=50)
sc.pp.neighbors(haensel_2021_ctrl_wounding_fb, use_rep='X_pca_harmony', n_neighbors=int(len(haensel_2021_ctrl_wounding_fb) ** 0.5 // 7), metric='cosine')
tk.tl.triku(haensel_2021_ctrl_wounding_fb)

In [ ]:
sc.tl.umap(haensel_2021_ctrl_wounding_fb, min_dist=0.2, random_state=seed)

In [ ]:
sc.tl.leiden(haensel_2021_ctrl_wounding_fb, resolution=0.2, random_state=seed)

sc.tl.leiden(haensel_2021_ctrl_wounding_fb, resolution=2.4, random_state=seed)
sc.tl.leiden(haensel_2021_ctrl_wounding_fb, resolution=5, random_state=seed)

In [ ]:
sc.pl.umap(haensel_2021_ctrl_wounding_fb, color=['leiden', 'Internal sample identifier', 'Internal sample identifier'], cmap=magma, use_raw=False, ncols=2)

In [ ]:
sc.pl.umap(haensel_2021_ctrl_wounding_fb, color=['leiden', 'Internal sample identifier', 'Pdgfra', 'Lum', 'Dcn'], legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

## Ma et al. 2020 [IT IS RAT, AND THE OVERLAP IS POOR]


In [ ]:
ma_dir = data_dir + '/ma_2020'

In [ ]:
adata_ma_Y = sc.read(f"{ma_dir}/adata_ma_Y.h5")

In [ ]:
# Basic QC filtering
adata_ma_Y.var['mt'] = adata_ma_Y.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_ma_Y, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(adata_ma_Y, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(adata_ma_Y, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_ma_Y, x='total_counts', y='n_genes_by_counts')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': adata_ma_Y.obs['batch'], 'y': adata_ma_Y.obs['log1p_n_genes_by_counts']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
adata_ma_Y = adata_ma_Y[(
                              ((adata_ma_Y.obs.batch == '0') & (adata_ma_Y.obs.log1p_n_genes_by_counts < 8.2) & 
                                (adata_ma_Y.obs.log1p_n_genes_by_counts > 7.3)) | 
                              ((adata_ma_Y.obs.batch == '1') & (adata_ma_Y.obs.log1p_n_genes_by_counts < 8.2) & 
                                (adata_ma_Y.obs.log1p_n_genes_by_counts > 7.3))).values, :]

In [ ]:
sc.pp.filter_genes(adata_ma_Y, min_counts=1)
sc.pp.normalize_total(adata_ma_Y)
sc.pp.log1p(adata_ma_Y)

In [ ]:
sc.pp.pca(adata_ma_Y, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_ma_Y, key='batch', max_iter_harmony=50)
sc.pp.neighbors(adata_ma_Y, use_rep='X_pca_harmony', n_neighbors=int(0.5 * len(adata_ma_Y) ** 0.5), metric='cosine')
tk.tl.triku(adata_ma_Y)

In [ ]:
sc.tl.umap(adata_ma_Y, min_dist=0.4, random_state=seed)

In [ ]:
sc.tl.leiden(adata_ma_Y, resolution=0.3, random_state=seed)

In [ ]:
assign_cats(adata_ma_Y, dict_cats=dict_cats_fb, min_score=0.5, quantile_gene_sel=0.9)
sc.pl.umap(adata_ma_Y, color=['leiden', 'batch', 'assigned_cats', 'Pdgfra', 'Lum', 'Dcn', ], legend_loc='on data', cmap=magma, ncols=2, 
           use_raw=False, )

In [ ]:
adata_ma_Y_fb = adata_ma_Y[adata_ma_Y.obs['assigned_cats'].isin(['fibro'])]

In [ ]:
sc.pp.filter_genes(adata_ma_Y_fb, min_counts=1)

In [ ]:
sc.pp.pca(adata_ma_Y_fb, random_state=seed, n_comps=25)
sce.pp.harmony_integrate(adata_ma_Y_fb, key='batch', max_iter_harmony=50)
sc.pp.neighbors(adata_ma_Y_fb, use_rep='X_pca_harmony', n_neighbors=int(len(adata_ma_Y_fb) ** 0.5 // 3), metric='cosine')
tk.tl.triku(adata_ma_Y_fb)

In [ ]:
sc.tl.umap(adata_ma_Y_fb, min_dist=0.25, random_state=seed)

In [ ]:
sc.tl.leiden(adata_ma_Y_fb, resolution=1.5, random_state=seed)
# sc.tl.leiden(adata_ma_Y_fb, resolution=0.15, random_state=seed)

In [ ]:
sc.pl.umap(adata_ma_Y_fb, color=['leiden', 'batch'], 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

In [ ]:
sc.pl.umap(adata_ma_Y_fb, color=['leiden', 'batch', 
'Tnfaip6', 'Ptx3', 'Cxcl1', 'Has1', 'Myoc', 'Elf1', 'Tfpi2', 'Tes',

                                ], legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
sc.tl.rank_genes_groups(adata_ma_Y_fb, groupby='leiden', method='wilcoxon')

In [ ]:
sc.pl.umap(adata_ma_Y_fb, color=['leiden', 'batch'] + list(adata_ma_Y_fb.uns['rank_genes_groups']['names']['20'][:250]), legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

## Phan 2020

In [ ]:
phan_2020_dir = data_dir + '/phan_2020'

In [ ]:
phan_2020_ctrl_21d = sc.read(f"{phan_2020_dir}/phan_2020_ctrl_21d.h5")

In [ ]:
# Basic QC filtering
phan_2020_ctrl_21d.var['mt'] = phan_2020_ctrl_21d.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(phan_2020_ctrl_21d, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(phan_2020_ctrl_21d, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(phan_2020_ctrl_21d, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(phan_2020_ctrl_21d, x='total_counts', y='n_genes_by_counts')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': phan_2020_ctrl_21d.obs['Internal sample identifier'], 'y': phan_2020_ctrl_21d.obs['log1p_n_genes_by_counts']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': phan_2020_ctrl_21d.obs['Internal sample identifier'], 'y': phan_2020_ctrl_21d.obs['pct_counts_mt']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
phan_2020_ctrl_21d = phan_2020_ctrl_21d[(
                              ((phan_2020_ctrl_21d.obs['Internal sample identifier'] == 'P21_Un_1') & (phan_2020_ctrl_21d.obs.log1p_n_genes_by_counts < 8) & 
                                (phan_2020_ctrl_21d.obs.log1p_n_genes_by_counts > 6.2)) | 
                              ((phan_2020_ctrl_21d.obs['Internal sample identifier'] == 'P21_Un_2') & (phan_2020_ctrl_21d.obs.log1p_n_genes_by_counts < 8) & 
                                (phan_2020_ctrl_21d.obs.log1p_n_genes_by_counts > 6.2)) | 
                              ((phan_2020_ctrl_21d.obs['Internal sample identifier'] == 'P21_Un_3') & (phan_2020_ctrl_21d.obs.log1p_n_genes_by_counts < 8) & 
                                (phan_2020_ctrl_21d.obs.log1p_n_genes_by_counts > 6.2)) 
                             ).values, :]
phan_2020_ctrl_21d = phan_2020_ctrl_21d[phan_2020_ctrl_21d.obs.pct_counts_mt < 5, :]

In [ ]:
sc.pp.filter_genes(phan_2020_ctrl_21d, min_counts=1)
sc.pp.normalize_total(phan_2020_ctrl_21d)
sc.pp.log1p(phan_2020_ctrl_21d)

In [ ]:
sc.pp.pca(phan_2020_ctrl_21d, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(phan_2020_ctrl_21d, key='Internal sample identifier', max_iter_harmony=50)
sc.pp.neighbors(phan_2020_ctrl_21d, use_rep='X_pca_harmony', n_neighbors=int(0.5 * len(phan_2020_ctrl_21d) ** 0.5), metric='cosine')
tk.tl.triku(phan_2020_ctrl_21d)

In [ ]:
sc.tl.umap(phan_2020_ctrl_21d, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(phan_2020_ctrl_21d, resolution=2, random_state=seed)

In [ ]:
assign_cats(phan_2020_ctrl_21d, dict_cats=dict_cats_fb, min_score=0.5)
sc.pl.umap(phan_2020_ctrl_21d, color=['leiden', 'Internal sample identifier', 'assigned_cats', 'Pdgfra', 'Lum', 'Dcn', 'Mpz', 'Plp1', 'Sfrp5', 'Chil1'], legend_loc='on data', cmap=magma, ncols=2, 
           use_raw=False, )

In [ ]:
phan_2020_ctrl_21d_fb = phan_2020_ctrl_21d[phan_2020_ctrl_21d.obs['assigned_cats'].isin(['fibro', 'fibro_2'])]

In [ ]:
sc.pp.filter_genes(phan_2020_ctrl_21d_fb, min_counts=1)

In [ ]:
sc.pp.pca(phan_2020_ctrl_21d_fb, random_state=seed, n_comps=25)
sce.pp.harmony_integrate(phan_2020_ctrl_21d_fb, key='Internal sample identifier', max_iter_harmony=50)
sc.pp.neighbors(phan_2020_ctrl_21d_fb, use_rep='X_pca_harmony', n_neighbors=int(len(phan_2020_ctrl_21d_fb) ** 0.5 // 4), metric='cosine')
tk.tl.triku(phan_2020_ctrl_21d_fb)

In [ ]:
sc.tl.umap(phan_2020_ctrl_21d_fb, min_dist=0.45, random_state=seed)

In [ ]:
sc.tl.leiden(phan_2020_ctrl_21d_fb, resolution=9, random_state=seed)

In [ ]:
sc.pl.umap(phan_2020_ctrl_21d_fb, color=['leiden',  'Internal sample identifier'], 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
sc.tl.rank_genes_groups(phan_2020_ctrl_21d_fb, groupby='leiden', method='wilcoxon')

In [ ]:
sc.pl.umap(phan_2020_ctrl_21d_fb, color=['leiden', 'Eln', 'Pi16', 'Nrep', 'Gas1', 'Cilp', 'Osr1', 'Hmcn1', 'Mmp16', 'Slit2', 'Ccl19', 'Tspan11', 'Rnf112', 'Col8a1', 'Tspan18', 'Il15', 'Mme', 'Col6a6', 'Fam69a',] , 
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

## Shook 2020

In [ ]:
shook_2020_dir = data_dir + '/shook_2020'

In [ ]:
shook_2020_ctrl = sc.read(f"{shook_2020_dir}/shook_2020_ctrl.h5")

In [ ]:
# Basic QC filtering
shook_2020_ctrl.var['mt'] = shook_2020_ctrl.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(shook_2020_ctrl, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(shook_2020_ctrl, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(shook_2020_ctrl, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(shook_2020_ctrl, x='total_counts', y='n_genes_by_counts')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': shook_2020_ctrl.obs['Internal sample identifier'], 'y': shook_2020_ctrl.obs['log1p_n_genes_by_counts']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': shook_2020_ctrl.obs['Internal sample identifier'], 'y': shook_2020_ctrl.obs['pct_counts_mt']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
shook_2020_ctrl = shook_2020_ctrl[(
#                               ((shook_2020_ctrl.obs['Internal sample identifier'] == '1') & (shook_2020_ctrl.obs.log1p_n_genes_by_counts < 7.3) & 
#                                 (shook_2020_ctrl.obs.log1p_n_genes_by_counts > 6.2)) | 
#                               ((shook_2020_ctrl.obs['Internal sample identifier'] == '2') & (shook_2020_ctrl.obs.log1p_n_genes_by_counts < 7.3) & 
#                                 (shook_2020_ctrl.obs.log1p_n_genes_by_counts > 6.2)) | 
                              ((shook_2020_ctrl.obs['Internal sample identifier'] == 'Ctrl_S4') & (shook_2020_ctrl.obs.log1p_n_genes_by_counts < 8) & 
                                (shook_2020_ctrl.obs.log1p_n_genes_by_counts > 7.35)) | 
                              ((shook_2020_ctrl.obs['Internal sample identifier'] == 'Ctrl_S5') & (shook_2020_ctrl.obs.log1p_n_genes_by_counts < 8) & 
                                (shook_2020_ctrl.obs.log1p_n_genes_by_counts > 7.35))).values, :]
shook_2020_ctrl = shook_2020_ctrl[shook_2020_ctrl.obs.pct_counts_mt < 7, :]

In [ ]:
sc.pp.filter_genes(shook_2020_ctrl, min_counts=1)
sc.pp.normalize_total(shook_2020_ctrl)
sc.pp.log1p(shook_2020_ctrl)

In [ ]:
sc.pp.pca(shook_2020_ctrl, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(shook_2020_ctrl, key='Internal sample identifier', max_iter_harmony=50)
sc.pp.neighbors(shook_2020_ctrl, use_rep='X_pca_harmony', n_neighbors=int(0.5 * len(shook_2020_ctrl) ** 0.5), metric='cosine')
tk.tl.triku(shook_2020_ctrl)

In [ ]:
sc.tl.umap(shook_2020_ctrl, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(shook_2020_ctrl, resolution=2, random_state=seed)

In [ ]:
assign_cats(shook_2020_ctrl, dict_cats=dict_cats_fb, min_score=0.45)
sc.pl.umap(shook_2020_ctrl, color=['leiden', 'Internal sample identifier', 'assigned_cats', 'Rgs5', 'Pecam1', 'Lyve1', 'Mlana'], legend_loc='on data', cmap=magma, ncols=2, 
           use_raw=False, )

In [ ]:
shook_2020_ctrl_fb = shook_2020_ctrl[shook_2020_ctrl.obs['assigned_cats'].isin(['fibro', 'fibro_2'])]

In [ ]:
sc.pp.filter_genes(shook_2020_ctrl_fb, min_counts=1)

In [ ]:
sc.pp.pca(shook_2020_ctrl_fb, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(shook_2020_ctrl_fb, key='Internal sample identifier', max_iter_harmony=50)
sc.pp.neighbors(shook_2020_ctrl_fb, use_rep='X_pca_harmony', n_neighbors=int(len(shook_2020_ctrl_fb) ** 0.5 // 4), metric='cosine')
tk.tl.triku(shook_2020_ctrl_fb)

In [ ]:
sc.tl.umap(shook_2020_ctrl_fb, min_dist=0.25, random_state=seed)

In [ ]:
sc.tl.leiden(shook_2020_ctrl_fb, resolution=7, random_state=seed)
# sc.tl.leiden(shook_2020_ctrl_fb, resolution=0.4, random_state=seed)

In [ ]:
sc.pl.umap(shook_2020_ctrl_fb, color=['leiden', 'Internal sample identifier',], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

In [ ]:
sc.tl.rank_genes_groups(shook_2020_ctrl_fb, groupby='leiden', method='wilcoxon')

In [ ]:
sc.pl.umap(shook_2020_ctrl_fb, color=['leiden', 'Internal sample identifier',] + list(shook_2020_ctrl_fb.uns['rank_genes_groups']['names']['53'][:300]), legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

## Determining overlap between populations

In [ ]:
dict_abasi_axis = {
                    'A': ['Akr1c18', 'Gap43', 'Ackr3', 'Ptn', 'Il18', 'Nrk', 'Efhd1', 'Wnt2', 'Aldh1a3', 'Cotl1', 'Gan', 'Pla1a', 'Lurap1l', 'Crispld1', 'Mustn1', 'Car8', ] + 
                        ['Pi16', 'Smpd3', 'Sfrp2', 'Plac8', 'Dpp4', 'Ifi27l2a', 'Sema3c', 'Serpinb6a', 'Creb5', 'Igfbp4', 'Pgr', 'Akr1c14', 'Ptgis', 'Dcbld2', 'Bmp7', 'Pcdh20', 'Slc16a1', ],
                    'B': ['Cygb', 'G0s2', 'Fzd4', ] + 
                        ['Col4a1', 'Mgp', 'Col4a2', 'Svep1', 'Fxyd6', 'F3', 'Ntrk2', 'Abca8a', ],
                    'C': ['Ndufa4l2', 'Serpina3n', 'Tnmd', ] + 
                        ['Cldn10', 'Ppp1r14a', 'Il1r2', 'Adh7', ],
                    'D': ['Ifitm1', 'a', 'Wif1', 'Tpm2', 'Slc6a6', 'Tns3', 'Inhba', 'Nav2', 'Aqp1', ] + 
                        ['Ncam1', 'Zfos1', 'Mef2c', 'Bcl2', 'Runx1', ],

}

dict_abassi = {
                    'A1': ['Stmn1', 'Cks2', 'Birc5', 'Lockd', 'Top2a', 'Ccna2', 'Ube2c', 'Diaph3', 'Cdca3', 'Spc25', 'Tpx2', 'Mki67', 'Pbk', 'Prc1', 'Cdca8', 'Cdk1', 'Hist1h2ap', 'Tacc3', 'Racgap1', 'Mxd3', 'Cdkn3', 'Nusap1', 'Ccnb1', 'Tk1', 'Kif11', 'Cenpe', 'Kif22', 'Ckap2l', 'Casc5', 'Foxm1', ] +
    ['Akr1c18', 'Gap43', 'Ackr3', 'Ptn', 'Il18', 'Nrk', 'Efhd1', 'Wnt2', 'Aldh1a3', 'Cotl1', 'Gan', 'Pla1a', 'Lurap1l', 'Crispld1', 'Mustn1', 'Car8', ] +
                        ['H2afz', 'Tuba1b', 'Hmgb2', 'Hmgb1', 'Smc4', 'Tmpo', 'Smc2', 'Rad21', 'Ccnb2', 'Cenpw', 'Lmnb1', 'Spc24', 'Cenpa', 'Cenpq', 'Cenpf', 'Cdc20', 'Ckap2', 'Pmf1', 'Tyms', 'Shcbp1', 'Cenpm', 'Knstrn', 'Fam64a', 'Aurkb', 'Mad2l1', 'Gmnn', 'Melk', 'Cdkn2c', 'Bub1', 'Ska1', 'Hmmr', 'Nuf2', 'Asf1b', 'Dbf4', 'Aspm', 'Bub1b', 'Ncapg2', ],
                    'A2': ['Pla1a', 'Cdc42ep2', 'Pamr1', 'Hebp2', ] + 
    ['Akr1c18', 'Gap43', 'Ackr3', 'Ptn', 'Il18', 'Nrk', 'Efhd1', 'Wnt2', 'Aldh1a3', 'Cotl1', 'Gan', 'Pla1a', 'Lurap1l', 'Crispld1', 'Mustn1', 'Car8', ] +
                        ['Wnt2', 'Lurap1l', 'Pcdh20', 'Ptpn5', 'Mustn1', 'Lims2', 'Wnt2', 'Bmyc', 'C1qtnf1', 'Pts', 'Kcnj8', 'Ankrd46', 'Sbsn', 'Slc38a4', 'Hdac7', 'Il33', 'Serpina3c', 'Mettl20', 'Pcdh20', 'Ptpn5', ],
                    'A3': ['Has2', 'Trib1', 'Fosl1', ] + 
    ['Akr1c18', 'Gap43', 'Ackr3', 'Ptn', 'Il18', 'Nrk', 'Efhd1', 'Wnt2', 'Aldh1a3', 'Cotl1', 'Gan', 'Pla1a', 'Lurap1l', 'Crispld1', 'Mustn1', 'Car8', ] +
                        ['Ccl2', 'Has1', 'Ptx3', 'Gm12840', 'Gfpt2', 'Ptgs2', 'Hmox1', 'Hk2', 'Mt2', 'Dcbld2', 'Cd44', 'Glul', 'Txnrd1', 'Slc16a1', 'Abl2', 'Baiap2', 'Dusp10', 'B4galt5', 'Srxn1', 'Sbsn', 'Slc38a4', 'Tnfrsf1b', 'Net1', 'Procr', 'Pcdh20', 'Fbxo30', 'Krtdap', 'Arl4d', 'Ptpn5', 'Cish', 'Aif1l', ],
                    'B1': ['Vwa1', 'Trf', 'Ngfr', 'Gcg', 'Sdc3', 'Col26a1', 'Tbata', 'Plxdc1', 'Ccl9', 'Shisa3', ] + 
    ['Cygb', 'G0s2', 'Fzd4', ] + 
                        ['Matn2', 'Vit', 'Cp', 'Apod', 'P2ry14', 'Pdrg1', ],
                    'B2': ['C2', 'Steap4', 'Osr1', 'Meox2', 'Cfb', 'Gdf10', ] + 
    ['Cygb', 'G0s2', 'Fzd4', ] + 
                        ['Mmp3', 'Igfbp7', 'Cygb', 'Vit', 'C1ra', 'Cp', 'Igfbp4', 'Plac8', 'Nr2f2', 'Tspan11', 'Serpinb1a', 'Ube2l6', 'F3', 'Chrdl1', 'Arhgdib', 'Ctsf', 'Naalad2', 'Scn7a', 'Cfh', ],
                    'B3': ['C2', 'Steap4', 'C3', 'C4b', 'Il6', 'Cfb', 'Bmper', 'Gdf10', 'Casp4', ] + 
    ['Cygb', 'G0s2', 'Fzd4', ] + 
                        ['Igfbp7', 'Apoe', 'Sepp1', 'Vit', 'C1ra', 'Cp', 'Igfbp4', 'Kitl', 'Osmr', 'Ifi203', 'Ube2l6', 'Tmem176a', 'Il33', 'Cxcl1', 'Tmem176b', ],
                    'B4': ['Wisp2', 'Megf6', 'Fgf9', ] + 
    ['Cygb', 'G0s2', 'Fzd4', ] + 
                        ['Igf1', 'Cilp', 'Eln', 'Sod3', 'Fbn2', 'Col12a1', 'Adamtsl2', 'Fxyd6', 'Sfrp4', 'Thbs4', 'Sema5a', 'Figf', 'Crispld2', 'Itgb5', 'Tgm2', 'Col8a1', 'Fibin', 'P4ha3', 'Cfh', 'Ptprd', 'Loxl3', ],
                    'B5': ['Thbs4', 'Fbln7', 'Meox1', 'Tnc', ] + 
    ['Cygb', 'G0s2', 'Fzd4', ] + 
                        ['Hmcn2', 'Col8a1', 'Fibin', 'Prss12', 'Dlk1', 'Adamtsl2', 'Hsd11b1', 'Ret', ],
                    'B6': ['Il1r2', 'Nov', 'Cd36', 'Gdf15', 'Rassf2', 'Cyp2f2', 'Akr1cl', 'Ddit4l', ] + 
    ['Cygb', 'G0s2', 'Fzd4', ] + 
                        ['Cxcl12', 'Pltp', 'Cdkn1c', 'Adam23', 'Tgfb2', 'Ccl19', 'Fabp4', 'Sectm1a', 'Adam12', 'Zfp385a', 'Oxtr', 'Tmem106a', 'Crp', ],
                    'C1': ['Akr1cl', 'Ddit4l', ] + 
    ['Ndufa4l2', 'Serpina3n', 'Tnmd', ] + 
                        ['Oxtr', 'Tmem106a', 'Crp', 'Gpc4', 'Cyp2f2', 'Npy1r', 'Ptgfr', 'Tgm2', 'Tmem106a', ],
                    'C2': ['Ndufa4l2', 'Cdh4', 'Emid1', 'Gpm6a', 'Igfbp2', ] + 
    ['Ndufa4l2', 'Serpina3n', 'Tnmd', ] + 
                        ['Cldn10', 'Ppp1r14a', 'Sulf2', 'S100a4', 'Ramp1', 'Osr2', 'Tnmd', 'Gal', 'Bcl11b', 'Ptk7', 'Fam105a', 'Ppp2r2c', 'Ccnd1', 'Cyp4b1', 'Camk2a', 'Fgfr4', 'Cpxm2', 'Ltbp2', 'Rspo1', 'Fam46c', 'Adcy1', 'Prkcb', 'Sema3b', 'Axin2', 'Tnfrsf19', 'Miat', 'Crabp1', 'Fcgr3', 'Wisp1', 'Abcc4', 'Rgcc', 'Lrx5', 'Arhgef19', 'Aldh3a1', 'Kif26a', 'Rassf4', 'Col7a1', ],
                    'C3': ['Emid1', 'Igfbp2', 'Stc1', ] + 
    ['Ndufa4l2', 'Serpina3n', 'Tnmd', 'Ahrr', 'Col7a1', 'Igfbp2', 'Rgcc', 'Rspo1', 'Wisp1',  ],
                    'C4': ['Coch', 'Fam180a'] + 
                          ['Cd9', 'Mfap4', 'Emp3', 'Eln', 'Eif4a2', 'Ramp1', 'Ccl19', 'Cyp26b1', 'Ackr4', 'Flt3l', 'Pi16', 'Il16', 'Ednra', ],
                    'D1': ['Mylk', 'Col11a1', 'Stmn2', 'Hmgcs2', 'Lrrc15', 'Tagln', 'Cox4i2', ] + 
    ['Ifitm1', 'a', 'Wif1', 'Tpm2', 'Slc6a6', 'Tns3', 'Inhba', 'Nav2', 'Aqp1', ] + 
                        ['Ifitm1', 'Ramp1', 'Eva1b', 'Wif1', 'Tpbg', 'Cd200', 'Hspb2', 'Acta2', 'Egfl6', 'Nradd', 'Aqp1', ],
                    'D2': ['Prlr', 'Rasd1', 'Serpine2', 'Inhba', 'Lef1', 'Tfap2a', 'Cpne5', 'Rspo3', 'Col23a1', 'Tfap2c', 'Dnm3os', 'Col13a1', 'Adrb3', 'Eva1a', 'Irf8', 'Slc12a1', 'Srcin1', 'Slc43a2', 'Wnt6', 'Alx4', 'Dgldn', 'Lamc3', 'Notum', 'Tcf7', 'Wnt10a', ] + 
    ['Ifitm1', 'a', 'Wif1', 'Tpm2', 'Slc6a6', 'Tns3', 'Inhba', 'Nav2', 'Aqp1', ] + 
                        ['Cox6b2', 'Enpp2', 'Crabp1', 'Trps1', 'Bcl2', 'Bmp7', 'Ctbp2', 'Ier2', 'Pappa2', 'Dkk2', 'Nrp2', 'Kdm7a', 'Rarres1', 'Pde4d', 'Auts2', 'Ldlrad3', 'Ackr4', 'Ptk7', 'Arhgap31', 'Ahdc1', 'Nav2', 'Unc5b', 'Sobp', 'Tanc1', 'Pgf', 'Scarb1', 'Dtx3', 'Ankrd11', 'Exoc6b', 'Notch1', ],
              }

dict_buechler_axis = {
                        'A': [] + 
                             [],
                    'B': ['Ccn2', 'Thbs4', 'Meox1', 'Adamtsl2', 'Tmem204', ] + 
                        ['Mgp', 'Cxcl14', 'Mfap4', 'Slit2', ],
                        'C': ['Nrp2', 'Axin2', ] + 
                        ['Cpz', 'Mmp27', 'Scpep1', 'Marveld1', 'Bmp4', 'Inhbb', 'Robo1', 'Mettl24', 'Oxtr', 'Fgfr2', 'Rtn4r', 'Zfp536', 'Cyp2f2', 'Serpina3n', 'Ppp2r2c', 'Tmem132c', 'Rasa3', 'Ddah1', 'Bcl11b', ],
                    'D': [] + 
                        ['Tpm2', 'Cd200', ],

}
dict_buechler = {
                    'A1': ['Aldh1a3', 'Fosl1', 'Ppp2r2b', 'Aif1l', 'Nr4a3', 'Dusp10', 'Tmem229a', ] + 
                        ['Ccl2', 'Akr1c18', 'Cxcl1', 'Ptgs2', 'Ppa1', 'Has1', 'Csrnp1', 'Maff', 'Il18', 'Ptx3', 'Wnt2', 'Gan', 'Procr', 'Limch1', 'Il33', 'Tubb4a', 'Glce', 'Wnt10b', 'Chst1', 'Dot1l', 'Sbsn', 'Adora2b', 'Pgr', 'Pvr', 'Gm20186', 'Nfkbib', 'Gm29408', 'Krt19', 'Bmp2', 'Krtdap', 'Anxa3', 'Pi16', 'Il33', 'Pgr', 'Chst1', 'Bmp7', ],
                    'A2': ['Arl4d', ] + 
                        ['Anxa3', 'Pi16', 'Efhd1', 'Wnt2', 'Aldh1a3', 'Limch1', 'Il33', 'Sbsn', 'Wnt10b', 'Fgf2', 'Pgr', 'Chst1', 'Bmp7', 'Dmkn', 'Krtdap', 'Aif1l', 'Igfbp4', 'Sytl2', 'Gpc6', ],
                    'A3': ['Hoxa9', 'Col6a6', ] + 
                        ['Cpxm1', 'Apod', 'Cxcl12', 'Svep1', 'Txnip', 'Col3a1', 'Smoc2', 'Ctsk', 'Entpd2', 'Lsp1', 'Colec12', 'Ogt', 'Mfap2', 'Matn2', 'Fap', 'Steap4', 'Snhg11', 'Adamts12', 'Itga11', 'Fads2', 'Gm44417', 'Adamts15', 'Gpm6b', ],
                    'A4': ['Apol9a', 'Apol9b', 'Ifi213', 'Oas3', 'Oas2', 'Ifi44', 'Dhx58', 'Sdc3', ] + 
                        ['Plac8', 'Bst2', 'Ly6e', 'Lgals3bp', 'Ifit3', 'Cxcl13', 'Irf7', 'Oasl2', 'Mndal', 'Ifit3b', 'Pycard', 'Gbp3', 'Tor3a', 'Stat1', 'Mx1', 'Cpt1a', ],
                    'A5': ['Ngfr', ] + 
                        ['Il6', 'Cxcl1', 'Ptx3', 'Ccl2', 'Has1', 'Gm48942', 'Ptgs2', 'Tnfaip3', 'Sod2', 'Ccn5', 'Thbs1', 'Itpkc', 'Cxcl10', 'Pgr', 'Tgm2', 'Has2', 'Cyp7b1', 'Trdn', ],
                    'B1': ['Adgre5', 'Col8a1', 'Nrep', 'Ret', ] + 
    ['Ccn2', 'Thbs4', 'Meox1', 'Adamtsl2', 'Tmem204', ] + 
                        ['Myoc', 'Hmcn2', 'Col5a3', 'Col4a2', 'Col6a3', 'Col4a1', 'Sparcl1', 'Prss12', 'Col6a6', 'Lamc1', 'Cmklr1', 'Msx1', 'G0s2', ],
                    'B2': ['Nkain4', 'Ecrg4', 'Tns3', 'S100g', 'Nrtn', 'Fgf9', 'Wfdc1', 'Angpt4', 'Klf5', 'Actn1', 'Cpxm2', 'Angptl7', 'Echdc2', 'Amigo2', 'Paqr6', 'Cracr2b', 'Npr3', 'Plxna4', 'Mlip', 'Ccdc3', 'Col8a2', ] + 
    ['Ccn2', 'Thbs4', 'Meox1', 'Adamtsl2', 'Tmem204', ] + 
                        ['Myoc', 'Hmcn2', 'F3', 'Igf1', 'F3', 'Ccn5', 'Arhgdib', 'Clu', 'Npy1r', 'Col12a1', 'Lepr', 'Grem2', 'Sfrp2', 'Sema5a', 'Gm5084', 'Id1', 'Ptprd', 'Rerg', 'Cobl', 'Sgcd', 'Cited1', 'Pappa2', 'Synpo', 'Kcnip4', 'Tmtc1', 'Ednra', 'Lgr5', 'Fmod', 'Gna14', 'Cobll1', ],
                    'B3': ['Tmem176a', 'Meox2', 'Vtn', 'Nr2f2', 'Tifa', 'Plvap', 'Ntf3', 'Slc1a3', 'Rgs16', 'Mlana', 'Talgps2', 'Dtx4', ] + 
    ['Ccn2', 'Thbs4', 'Meox1', 'Adamtsl2', 'Tmem204', ] + 
                        ['Ifit3', 'Ifit1', 'Cxcl13', 'Irgm1', 'Igfbp7', 'Apoe', 'Tmem176b', 'Sparcl1', 'Cfh', 'Steap4', 'F3', 'Vit', 'Penk', 'Cxcl14', 'Ccl19', 'Cyp2f2', 'Cp', 'Adcyap1r1', 'Mx2', 'Spock2', 'Adamtsl3', 'Srgn', 'Tnfsf13b', 'Bmper', 'Prg4', 'Aoc3', 'Nfasc', 'Lmo2', 'Lifr', 'Rab20', 'Ptch2', 'Gm10134', 'Dkk3', 'Sept4', 'Gata6', 'Gk5', 'Tmeff2', 'Mbd1', 'Isyna1', 'Arhgap15', 'Crocc2', ],
                    'B4': ['Mamdc2',]  + 
    ['Ccn2', 'Thbs4', 'Meox1', 'Adamtsl2', 'Tmem204', ] + 
                          ['Cyp2f2', 'Sparcl1', 'Galnt15', 'Cyp1b1', 'Penk', 'Angptl4', 'Fzd4', 'Mex3b', 'Col6a6', 'Peg3', 'Alpl', 'Lpar4', 'Agt', 'Cadm1', 'Aoc3', 'Thsd7a', 'Sept4', 'Tiam2', ],
                    'C1': ['Aldh3a1', 'Cdh4', 'Sphk1', 'Fgfr4', 'Grem1', 'Pla2g2e', 'Nt5e', 'Cgref1', 'Tnfrsf19', 'Pla2g5', 'Gdf15', 'Robo1', 'Pik3r6', 'Tent5c', 'Creb3l3', 'Miat', ] + 
                        ['S100a4', 'Ndufa4l2', 'Col16a1', 'Ppp1r14a', 'Cldn10', 'Cyp26b1', 'Rspo1', 'Tmem132c', 'Vit', 'Grem2', 'Fam180a', 'Gpm6a', 'Trps1', 'Wnt5a', 'Atp1a2', 'Kazald1', 'Slc27a6', 'Gm43042', 'Arhgap15', 'Ahr', 'Ahrr', 'Evi2a', 'Ccdc42', 'Cdr2', 'Sema3a', 'Camk2b', 'Nkd2', 'Ccbe1', 'Prex1', 'Arhgap25', ],
                    'C2': ['Masp1', 'Crp', 'Hmcn1', 'Akr1cl', ] + 
                        ['Pltp', 'Lepr', 'Sectm1a', 'Rtn4r', 'Lgr5', 'Col12a1', 'Hmcn1', 'Angptl4', 'Ptgfr', 'St6galnac5', 'Nkain4', 'Abcc9', 'Mkx', 'Rerg', 'Gng13', 'Gfra2', 'Iqgap2', ],
                    'D1': ['Lrrc15', 'Lamc3', 'Rasgrp2', 'Aqp1', 'Gli1', 'Acan', 'Foxd2os', 'Actg2', 'a', 'Cspg4', 'Ednrb', 'Adamts18', 'Alx4', 'Itga8', ] + 
                        ['Pfn2', 'Wif1', 'F2r', 'Col11a1', 'Tnmd', 'Trps1', 'Aopep', 'Planp', 'Actn1', 'Tagln', 'Stmn1', 'Myo1b', 'Bok', 'Rarres1', 'Acta2', 'Nav2', 'Igfbp2', 'Hck', 'Tenm3', 'Ednra', 'Stmn2', 'Sox18', 'Col7a1', 'Mafb', 'Egflam', 'Fabp5', 'Lmo1', 'Kif26b', 'Mylk', 'Cox4i2', 'Runx1', 'Cpne5', 'Tpd52l1', ],
                    'D2': ['Itgb4', 'Sfrp5', 'Kank4', 'Itga6', 'Hhip', 'Crabp1', 'Erbb3', 'Pcp4l1', 'Bcam', 'Mcam', ] + 
                        ['Ptch1', 'Ldhb', 'Tns3', 'Ptprz1', 'Trpm3', 'Nr2f2', 'Chodl', 'Chl1', 'Aspa', 'Gm27820', 'Gm48159', 'Cadm1', 'Cd24a', ],
}

dict_haensel_axis = {
                    'A': [] + 
                        ['Aldh3a1', 'Ankrd29', 'Cacna2d3', 'Cib3', 'Col1a1', 'Col1a2', 'Crocc', 'Cybrd1', 'Duoxa1', 'Fgfr4', 'Gale', 'Ndua4l2', 'Pla2g2e', 'Pla2g5', 'Prex1', 'Sparc', 'Tmem150', 'Zdhhc18', ],
                    'B': [] + 
                        ['Gpx3', 'Ogn', 'Cxcl12', 'Ifi205', 'Nfib', 'Pltp', 'Lbp', 'Cilp', 'Pi16', 'Anxa3', 'Serpinb6a', 'Cygb', 'Gpc3', 'Prss23', 'Sfrp1', 'Tmeff2', 'Gfpt2', 'Akr1c14', 'Gngt2', 'Mnda', 'C4b', 'Slc43a3', 'Ggt5', 'Il4ra', 'Crp', 'Zyx', 'Atp8b1', 'Tgm2', 'Col12a1'],
                    'C': [] + 
                        ['Cthrc1', 'Ptn', 'Cxcl1', 'Il1rl1', 'Arl4c', 'Lrrc15', 'Runx1', 'Adam12', 'Nt5dc2', 'Actn1', 'Cxcl5', 'Arl4c', 'Ccnb2', 'Birc5'],
                    'D': [] + 
                    ['Cd9', 'Dkk2', 'Gas1', 'Mafb', 'Enpp2', 'Coch', 'Slit2', 'Pianp', 'Eid1', 'Nrep', 'Csrp1', 'Pmepa1', 'Postn', 'Cnn2'],
                   'E': [] + 
                    ['Lgals7', 'Fxyd3', 'Perp3', 'S100a14', 'FKrt15', 'Krt5', 'Sfn', 'Apoe', 'Anxa8', 'Ccl27a', 'Col17a1', 'Dmkn', 'Krt14', 'Dsp', 'Trim29', 'Tacstd2', 'Urah', 'Dapl1', 'Apoc1', 'Spint2', 'Clca3a2', 'Gata3', 'Dsc3', 'Trp63', 'Capns2', 'Serpinb5', 'Pkp3', 'Net1', 'Rab25', 'Dsg1a', 'Fcgbp', 'Pou3f1', 'Pkp1', 'Itga6', 'Ptprf', 'Calm5', 'Cldn1', 'Irf6', 'Pkib'],
                   'F': [] + 
                    ['Fcer1g', 'Tyrobp', 'Srgn', 'Cd52', 'Laptm5', 'Il1b', 'Plek', 'Lilrb4', 'Marcksl1', 'Lcp1', 'Il1rn', 'Cd53', 'Cd14', 'Clec4d', 'Slfn2', 'Clec4e', 'Ucp2', 'Ctss', 'Lilr4b', 'Tpd52', 'Coro1a', 'Slc7a11', 'Mxd1', 'Ptafr', 'Wfdc17', 'Ccrl2', 'Nlrp3', 'Ptprc', 'Spi1', 'Slpi', 'Traf1', 'Rac2', 'Irg1', 'Bcl2a1b', 'Ccl6', 'Gmfg', 'S100a8', 'Dusp5', 'Slc15a3', 'Selplg', 'Smox', 'Ptpn18', 'Cd68']
}

dict_haensel = {
                   'A1': ['Dusp26', 'Thsd4', 'Lin7a', 'Cadm1', 'Poc1a', ] + 
                    ['Twist2', 'Col6a3', 'F13a1', 'Mme', 'Kcnk2', 'Stc1', 'Gldn', 'Rspo3', 'Rgs10', 'Cdkn2c', 'Col23a1', 'Rspo4', 'Pamr1', 'Enho', 'Syt13', 'Srpx2', 'Plxdc1', 'Daam2', 'Ackr2', 'Spock2', 'Fcgr2b', 'Ptgs1', 'Notum', 'Ntf3', 'Edn3', 'Ociad2', 'Stmn2', 'Bdnf', 'Smox', 'Cyp27a1', 'Prdm1', 'Cmklr1', 'Fam83a', 'Itga4', 'Sgip', 'Specc1', 'Chodl', 'Flnc', 'Ccbe1', ],
                   'A2': ['Moxd1', 'Ptchd4', ] + 
                    ['Igfbp2', 'Col1a1', 'Qpct', 'Stc1', 'Sema3a', 'Twist2', 'Grem1', 'Ramp1', 'Serpina3g', 'Ccbe1', 'Ackr4', 'Mamdc2', 'Slc6a2', 'Irx6', 'Col13a1', 'Adcy1', ],
                   'A3': ['Pla2g7', 'Col13a1', ] + 
                    ['Col1a1', 'Col1a2', 'Cgref1', 'Grem1', 'Miat', 'Mamdc2', 'Adcy1', 'Slc6a2', 'Gna14', 'Cybrd1', 'Notum', 'Cib3', 'Vstm2a', 'Cubn', 'Klhl14', 'Parm1', 'Pi15', 'Scg3', ],
                   'A4': ['Serpina3m', 'Gbp5', ] + 
                    ['Mt1', 'Tnfaip6', 'Mt2', 'Fgl1', 'Has1', 'Il6', 'Slc10a6', 'Ier5', 'Sphk1', 'Ets2', 'Casp4', 'Bcl3', 'Rnf122', 'Arid5a', 'Vit', 'Tnfaip3', 'Sowahc', 'Vcam1', 'Pim1', 'Ccl2', 'Csf1r', 'Amy1', 'Stk40', 'Sbno2', 'Aspg', 'Gch1', 'Gpm6a', 'Ripk1', 'Arl4d', 'Irs2', 'Trib1', 'Oxtr', 'Pla2g5', 'Ager', ],
                   'A5': ['Enc1', 'Serpina10', ] + 
                    ['Ccl19', 'Lgals3', 'Ccl8', 'Cpxm1', 'Ltbp1', 'Mmp3', 'H2-Q7', 'Pla2g2e', 'Tmem176a', 'Tmem176b', 'Ugcg', 'Cyp7b1', 'H2-Q6','Ass1', 'Ecscr', 'Wfdc12', 'Slamf8', 'Tnc', 'Chl1', 'Lif', 'Tmem150c', 'Hgf', 'Aspg', ],
                   'A6': [] + 
                    ['Ankrd29', 'Cib3', 'Crocc', 'Gale', 'Nxph4', 'Pla2g5', 'Zdhhc18', 'Prex1', 'Miat', 'Flrt1', 'Aspg', 'Ces2g', 'Pi15', 'Oxtr', 'Gpha2', 'Glt8d2', 'Amy1', 'Ankrd29', 'Rassf4', 'Pik3r6', 'Ahrr', 'Dnajc4', 'Cyp2f2', ],
                   'B1': [] + 
                    ['Cpxm1', 'Cxcl12', 'Gpx3', 'Tmem176a', 'Tmem176b', 'Thbs1', 'Pltp', 'Ggt5', 'Sned1', 'Chl1', 'Lbp', 'Tnc', 'Cyp7b1', 'Ace', 'Sfrp1', 'Col12a1', 'Lgr5', 'Ifi204', 'Ctla2a', 'Enc1', 'Lcn2'],
                   'B2': ['Crp', 'Akr1cl', 'Art3', 'Pgr', 'Alpl', 'Greb1'] + 
                    ['Gpx3', 'Dpep1', 'Gas6', 'Cilp', 'Igf1', 'Pltp', 'Ctsh', 'Anxa3', 'Akr1c14', 'Kcnj8', 'Pcsk6', 'Ctla2a', 'Nkain4', 'Abcc9', 'Greb1', 'Sema3d', 'Rerg', 'Slco2a1', 'Tgm2'],
                   'B3': ['Nkain4'] + 
                    ['Cxcl12', 'Sfrp1', 'Sectm1a', 'Mnda', 'Slc43a3', 'Kcnj8', 'Ifi203', 'Lgr5', 'Slfn5', 'Myoc', 'Pltp', 'Lbp', 'Sfrp1', 'Ifi205', 'Postn', 'Ifi204', 'Galnt15', 'Kcnj8'],
                   'B4': [] + 
                    ['Cxcl12', 'Postn', 'Cygb', 'Itih5', 'Ifi204', 'Gpc3', 'Nfib', 'Gfpt2', 'Col4a1', 'Ggt5', 'Pltp', 'Tmeff2', 'Plac8', 'Pde3a', 'Adam19', 'Palld', 'Peg3', 'Nrep', 'Adam23', 'Prss12', 'Tmem158', 'Col6a6', 'Cxcl13', 'Fzd4', 'Scn7a', 'Slco2a1', 'Fabp4', 'Tspan7', 'Gucy1b3', 'Gpc6', 'Hs3st1', 'Cpe', 'Itga11', 'Fbln7'],
                   'B5': ['Hc', 'Angpt2', 'Col13a1', 'Stra6', 'Nxph3', 'Prune2', 'S1pr1', 'Nes'] + 
                    ['Igfbp4', 'Itm2a', 'Postn', 'Rbp1', 'Pla1a', 'Mdk', 'Sfrp2', 'Col18a1', 'Wnt2', 'Fbln7', 'Ptgis', 'Inmt', 'Oaf', 'Palld', 'Ldhb', 'Mafb', 'Slc9a3r2', 'Cfh', 'Adam12', 'Spon1', 'Mme', 'Gria3', 'Mmd', 'Nxn', 'Ttyh2', 'Ptn', 'Nr2f2', 'Rednra', 'Pear1', 'Enpep', 'Tnfrsf21', 'Crabp1', 'Fhl2', 'As3mt', 'Igfs2', 'Runx2', 'Serpini1', 'Sbspon', 'Map2', 'Slc38a4', 'Vldlr', 'Batf3', 'Tbx2'],
                   'B6': ['Akr1c18', 'Smpd3', 'Cxcl13', 'Chrdl1', 'Car8', 'Phf11b', 'Prkg2', 'Limch1', 'Plscr2', 'Crispld1', 'Edn1', 'Bmp8a', 'Ppp2r2b', 'Adgrg6', 'Pcdh9'] + 
                   ['Gpx3', 'Plac8', 'Anxa3', 'C3', 'C4b', 'Nid1', 'Dpp4', 'Pla1a', 'Igfbp4', 'Ace', 'Tmeff2', 'Efhd1', 'Mustn1', 'Figf', 'Il18', 'Sfrp2', 'Tshz2', 'Cmah', 'Inmt', 'Tek', 'Wnt2', 'Aldh1a3', 'Akr1c14', 'Scn7a', 'Sp110', 'Nova1', 'Spon1', 'Pygl', 'Fbxw17', 'Plce1', 'Gabra3', 'Clip4', 'Pnp2', 'Tmod1', 'Rorb', 'Kcnk5', 'Irf4', 'Chst1', 'Ano3', 'Krt36', 'Pcdh11x'],
                   'B7': ['Adamtsl2', 'Qprt', 'Bmp5', 'Ret', 'Ces1d'] + 
                    ['Gpx3', 'Itih5', 'Mgp', 'Hmcn2', 'Thbs4', 'Myoc', 'Fap', 'Sfrp1', 'Fbln7', 'Prss12', 'Msx1', 'Pcsk6', 'Meox1', 'Fzd4', 'Col8a1', 'Col6a6', 'Mest', 'Inmt', 'Itga11', 'Ptgis', 'Bmper', 'Vtn', 'Gfra1', 'Vwa1', 'P2ry14', 'Sept4', 'Mn1', 'Ncald', 'Mgarp', 'Msc', 'Il17d'],
                   'B8': ['Gdf10'] + 
                    ['Gpx3', 'Cygb', 'Mgp', 'Steap4', 'Ace', 'Cfh', 'F3', 'C2', 'Bmper', 'Emb', 'Nid2', 'Fzd4', 'Lgi2', 'Vtn', 'Nr2f2', 'Saa3', 'Gabra3', 'Sept4', 'Mest', 'Fbln7', 'Fhl2', 'Nr2f2', 'Ccdc3', 'Ptch2', 'Naalad2', 'Rgs16'],
                   'B9': ['Sbspon', 'Gcg', 'Col28a1', 'Col26a1', 'Kank4', 'Nipal1', 'Sox8', 'Cdh19'] + 
                    ['Apod', 'Cxcl1', 'Cp', 'Cpe', 'Igfbp4', 'Rasgrp2', 'Ccl9', 'Trf', 'Vit', 'Cwa1', 'Gfra1', 'Ebf2', 'P2ry14', 'Vtn', 'Cspg4', 'Olfml2a', 'Sox9', 'Plxdc1', 'Adamtsl3', 'Foxsl1', 'Clec1a', 'Etv1', 'Traf4', 'Etv4', 'Gli1', 'Nr1h3', 'Klf15', 'Rcsd1', 'Sox5', 'Fam20c', 'Aspa', 'Acot1', 'Foxp2', 'Cdkn2a', 'Hlf', 'Matn4'],    
                   'C1': ['Gap43', 'Stmn4', 'Gata6', 'Atp1b1', 'Rbbp8', 'Rab27b', 'Ccdc68', 'Csf2', 'Lgi3', 'Spsb4'] + 
                    ['Fstl1', 'Ifi27l2a', 'Col5a3', 'Sfrp2', 'Cxcl1', 'Timp2', 'Ptx3', 'Ugdh', 'Rras2', 'Sbsn', 'Nov', 'Ptgis', 'Rcan1', 'Noct', 'H19', 'Flnb', 'Wnt2', 'Serpinb2', 'Sfrp4', 'Efhd1', 'Il1rl1', 'Rnd1', 'Plk2', 'Rab32', 'Dusp10', 'Pamr1', 'Ptgs1', 'Akr1c18', 'Serpine1', 'Cmah', 'Il18', 'Pde8a', 'Cxcl3', 'Ngf', 'Jag1', 'Slc38a4', 'Aldh1a3', 'Aif1l', 'Ccnb2', 'Cenpa', 'Sox11', 'Cdca3', 'Islr2'],
                   'C2': ['Lcn2', 'Gm15247', 'Lmcd1', 'Ereg'] +
                    ['Saa3', 'Il33', 'Timp1', 'Pcsk5', 'Serpinb2', 'Cxcl5', 'Ptx3', 'Prg4', 'P4ha3', 'Il1rl1', 'Chl1', 'Rnd1', 'Hp', 'Gm16685', 'Cxcl3', 'Depdc5', 'Ngf', 'Adarb1', 'Slc16a3', 'Cyp7b1', 'Epas1', 'Lsmem1', 'Steap1', 'Inhba', 'Gpc1'],
                   'C3': ['Kif26b', 'Fblim1', 'Tmem86a', 'Runx2'] + 
                    ['Postn', 'Tnc', 'Tpm2', 'Acta2', 'Rbp1', 'Adam12', 'P4ha3', 'Mdk', 'Nt5dc2', 'Cnn2', 'Pla2g16', 'Wisp1', 'Tfpi2', 'Spp1', 'Myh9', 'H19', 'Nek6', 'Fhl2', 'Col8a1', 'Csgalnact1', 'Cdca3', 'Aoc3', 'Capn6', 'Tln2', 'Uck2', 'Soat1', 'Knstrn', 'Rnf150', 'Kcne4'],

                   'D1': ['Pde4d', 'Tbx3', 'Chrm2', 'Amn', 'Hapln1', 'Aldh1a2', 'Sdk2'] +
                    ['Aqp1', 'Igfbp4', 'Spry1', 'Ass1', 'Runx1', 'Fcgr2b', 'Rspo4', 'Spon1', 'Meox2', 'Hsd17b7', 'Cbfa2t3', 'Slc5a3', 'Adgrl3', 'Foxd1', 'Itga8', 'Shc4', 'Epha7', 'Wfdc1', 'Nnat', 'Bpgm', 'Lrrn1', 'Tspan15', 'Map2'],
                   'D2': ['Kera', 'Plk5', 'Serpina10', 'Adamts9'] + 
                    ['Coch', 'Tnmd', 'Dkk2', 'Mafb', 'Gldn', 'Megf6', 'Col11a1', 'Fmod', 'Meox2', 'Ntn5', 'Hs3st6', 'Il15', 'Fbln7', 'Fcgr2b', 'Tnfrsf11b', 'Sapcd1', 'Nav3', 'Wfdc1', 'Rspo4', 'Kif26b', 'Edn3', 'Cib2', 'Hoxd9', 'Tspan7', 'Epha7', 'Chrna1', 'Gfra1'],
                   'D3': ['Wif1', 'Actg2', 'Nav2', 'Cspg4', 'Rasgrp2', 'Ptger3'] +
                    ['Vim', 'Tpm2', 'Myl9', 'a', 'Lrrc15', 'Enpp2', 'Bcl2', 'Cnn2', 'Tpbg', 'Tagln', 'Acta2', 'Tnn', 'Cox6b2', 'Igfbp4', 'Aqp1', 'Ptger3', 'Col11a1', 'Serpine2', 'Stmn2', 'Cd200', 'Tns3', 'Fam101b', 'Itga8', 'Nexn', 'Ntrk3', 'Gng2', 'Runx3', 'Sema3g'],
                   'D4': ['Clec14a', 'Trib2'] +
                    ['Mgp', 'Dkk2', 'Mafb', 'Ramp1', 'Eln', 'Fam180a', 'Pmepa1', 'Cpxm2', 'Phlda3', 'Id1', 'Ctgf', 'Myoc', 'Ednra', 'Pappa2', 'Plxdc1', 'F2r', 'Mmp16', 'Lgr5', 'Lamc3', 'P2ry1', 'Plekha6', 'Gng8', 'Mustn1', 'Rnf112', 'Baiap2', 'Enpp1', 'Chchd10', 'Col8a2', 'Tmem255b', 'Wnt5b'],
                   'D5': ['Tnfrsf21', 'Ramp3'] + 
                    ['Mfap4', 'Nrn1', 'Nrep', 'Ackr4', 'Cmklr1', 'Syt13', 'Il15', 'H2-Q6', 'Pde2a', 'Cbfa2t3', 'Kitl'],
                   'E': [] + 
                    ['Lgals7', 'Fxyd3', 'Perp3', 'S100a14', 'FKrt15', 'Krt5', 'Sfn', 'Apoe', 'Anxa8', 'Ccl27a', 'Col17a1', 'Dmkn', 'Krt14', 'Dsp', 'Trim29', 'Tacstd2', 'Urah', 'Dapl1', 'Apoc1', 'Spint2', 'Clca3a2', 'Gata3', 'Dsc3', 'Trp63', 'Capns2', 'Serpinb5', 'Pkp3', 'Net1', 'Rab25', 'Dsg1a', 'Fcgbp', 'Pou3f1', 'Pkp1', 'Itga6', 'Ptprf', 'Calm5', 'Cldn1', 'Irf6', 'Pkib'],
                   'F': [] + 
                    ['Fcer1g', 'Tyrobp', 'Srgn', 'Cd52', 'Laptm5', 'Il1b', 'Plek', 'Lilrb4', 'Marcksl1', 'Lcp1', 'Il1rn', 'Cd53', 'Cd14', 'Clec4d', 'Slfn2', 'Clec4e', 'Ucp2', 'Ctss', 'Lilr4b', 'Tpd52', 'Coro1a', 'Slc7a11', 'Mxd1', 'Ptafr', 'Wfdc17', 'Ccrl2', 'Nlrp3', 'Ptprc', 'Spi1', 'Slpi', 'Traf1', 'Rac2', 'Irg1', 'Bcl2a1b', 'Ccl6', 'Gmfg', 'S100a8', 'Dusp5', 'Slc15a3', 'Selplg', 'Smox', 'Ptpn18', 'Cd68']
}


dict_phan_axis = {
                    'A': ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                        ['Col3a1', 'Clec3b', 'Ly6a', 'Sulf2', 'Ctsh', 'P4ha2', 'Fam46a', 'Cldn10', 'Il1r2', 'Mmp3', 'Ccl11', 'Hpgd', 'Eln', ],

                    'B': ['Fmod', 'Kera', 'Fibin', 'Megf6', 'Sapcd1', 'Itga11', 'Nav3', 'Ntn5', 'Adamts9', ] + 
                        ['Coch', 'Tnmd', 'Col11a1', 'Crabp1', 'Emid1', 'Gldn', 'Srpx2', 'Gpm6b', 'Ifitm1', 'Phldb2', 'Col7a1', 'Kif26b', 'Itih5', 'Pcbd1', 'Chst15', 'Apela', ],

                    'C': [] + 
                        ['Slc6a6', 'Angptl4', 'Tns3', 'Gadd45a', ],

                    'D': ['Dusp6', 'Emilin3', 'Itga8', 'Nrp1', 'Pcdh7', 'Scml4', ] + 
                         ['Aqp1', 'Calcrl', 'Chst1', 'Col14a1', 'Col26a1', 'Cyth3', 'Ddr2', 'Efna5', 'Emp2', 'Igfbp4', 'Jade1', 'Thy1', 'Vcan', 'Zeb1', 'Col6a2', ],

                    'E': ['Chil1', 'Cldn1', 'Itga6', 'Klf5', ] + 
                         ['Bnc2', 'Clec1a', 'Ebf2', 'Itgb4', 'Krt19', 'Lbp', 'Pear1', 'Plce1', 'Saa1', 'Sbspon', 'Slc6a13', 'Tgm2', 'Tln2', 'Tubb2b', 'Igfbp6', 'Cav1', 'Fxyd6', 'Mgp', 'Csrp1', 'Tenm2', 'Ildr2', 'Mdfic', 'Gab1', 'Cav2', ],
                    'F': ['Mbp', 'Plp1', 'Gatm', 'Mpz', 'Kcna1', 'Cldn1', 'Mal', 'Egfl8', 'Gjc3', 'Ncmap', 'Pllp', 'Bcas1', 'Plekhb1', 'Kcna2', 'Mlip', 'Gfra3', 'Pou3f1', 'Ptpre', 'Gas2l3', ] + 
                        ['Cldn1', 'Ebf2', 'Itga6', 'Sbspon', 'Krt19', 'Art3', 'Vwa1', 'Cldn19', 'Aatk', 'Rasal2', 'S100b', 'Slc6a13', ],
    
}

dict_phan = {
                    'A1': ['Serpina3n', 'Creb3l3', 'Mmp27', 'Fcgr3', 'Bcat1', ] + 
    ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                        ['Pcolce2', 'Ppp1r14a', 'Cldn10', 'Cgref1', 'Il1r2', 'Angptl1', 'Gstm2', 'Tmem97', 'Ltbp2', 'Gda', 'Cgref1', 'C1qtnf3', 'St3gal1', 'Wisp2', 'Timp1', 'Basp1', 'BC048602', 'Casp12', 'Tcea3', 'Olfml1', 'Sema3c', 'Penk', 'Cyp4b1', 'Arhgef19', 'Adcy1', 'Aldh1l2', 'Adamtsl2', 'Bdh2', 'Cyp2f2', 'Ccl8', 'Arhgap20', ],
                    'A2': ['Cdh6', 'Grem1', 'F13a1', 'Scg3', ] + 
    ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                        ['Igfbp2', 'Thbs2', 'Mmp3', 'Wnt5a', 'Gda', 'Ccdc42', 'Fxyd6', 'Hmcn1', 'Mbd2', 'Grem2', 'Nov', 'Cdh13', ],
                    'A3': ['Mmp16', 'Rnf112', ] + 
    ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                        ['Eln', 'Pi16', 'Nrep', 'Gas1', 'Cilp', 'Osr1', 'Hmcn1', 'Slit2', 'Ccl19', 'Tspan11', 'Col8a1', 'Tspan18', 'Il15', 'Mme', 'Col6a6', 'Fam69a', ],
                    'A4': ['Tmeff2', 'Gpx3', 'Kcnj8', 'Ace', 'C3', 'Ifi205', 'Ptgs2', 'H19', 'Sfrp4', ] + 
    ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                        ['Dpt', 'Clec3b', 'Col14a1', 'C1ra', 'Pi15', 'Svep1', 'Cdon', 'Ablim1', 'Ly6a', 'Cygb', 'Pcolce2', 'Postn', 'Fndc1', 'Dapk1', 'Cadm2', 'Man2a1', 'Efemp1', 'Ms4a4d', 'Nova1', 'Gas6', 'Mgst1', 'Slc1a5', 'Fkbp11', 'Csf1', 'Hpgd', 'Heg1', 'Slc43a3', 'Lgr5', 'Pi15', ],
                    'A5': ['Dcbld2', 'P2ry14', 'Trf', ] + 
    ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                        ['Smoc2', 'Gpc3', 'Ccl11', 'Apod', 'Gpc3', 'Vit', 'Cxcl1', 'Cp', 'Ebf2', 'Vwa1', 'Sult5a1', 'Steap4', 'Thbs4', 'Sox9', 'Sdc3', 'Clec1a', 'Myoc', 'Vtn', 'Dlk1', 'Saa1', 'Ace', 'Il34', 'Col8a1', 'Atp8b1', 'Rasgrp2', 'Reln', 'Ptx3', 'Il6', 'Cldn1', ],
                    'A6': ['Tmeff2', 'F3', 'Vcam1', 'Rxfp1', 'Nfib', 'Gata6', ] + 
    ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                          ['Dpt', 'Clec3b', 'Col14a1', 'C1ra', 'Pi15', 'Svep1', 'Cdon', 'Ablim1', 'Igfbp3', 'Slfn5', 'Mt1', 'Pi15', 'Ccl2', 'Fbln1', 'F3', 'Col4a5', 'Abcc9', 'Fmo2', ],
                    'A7': [ ] + 
    ['Pcolce2', 'Ppp1r14a', 'Man1a', 'Creb5', 'Tnxb', 'Gstm2', ] + 
                          ['Dusp1', 'Atf3', 'Nfkbia', 'Osr2', 'Rhob', 'Wsb1', 'Ccl7', 'Osr2', 'Adamts15', 'Nfkb2', 'Baiap2', 'Ccl2', 'Bicra', 'Cxcl10', 'Btg1', 'Has2', 'Ccl19', 'Chd2', ],
                    'B1': ['Ngb', 'Kif26b', 'Galr2', 'Mllt3', 'Krt8', 'Sema3e', 'Gdnf', 'Fam213a', 'Pcdh20', 'Clstn2', 'Tspan6', 'Col24a1', 'Nol3', 'Kcnab1', 'Adgra1', 'Cck', ] +
    ['Fmod', 'Kera', 'Fibin', 'Megf6', 'Sapcd1', 'Itga11', 'Nav3', 'Ntn5', 'Adamts9', ] +
                        ['Fmod', 'Col7a1', 'Nav3', 'Itga11', 'Sapcd1', 'Edn3', 'Cpne5', 'Rspo4', 'St3gal5', 'Tmod3', 'Ngfr', 'Wif1', 'Wfdc1', 'Plxdc1', 'Pdlim5', 'Smim5', 'Tagln', 'Abca9', 'Spats2l', 'Omd', 'Car11', 'Jam3', 'Shisa4', ],
                    'B2': ['Chst15', ] + 
    ['Fmod', 'Kera', 'Fibin', 'Megf6', 'Sapcd1', 'Itga11', 'Nav3', 'Ntn5', 'Adamts9', ] + 
                        ['Prelp', 'Aspn', 'Cybrd1', 'Prss23', 'Daam2', ],
                    'B3': ['Col8a1', ] + 
    ['Fmod', 'Kera', 'Fibin', 'Megf6', 'Sapcd1', 'Itga11', 'Nav3', 'Ntn5', 'Adamts9', ] + 
                        ['Postn', 'Cpe', 'Dpep1', 'Fam105a', 'Igf1', 'Tspan11', 'Steap4', 'Sostdc1', 'Cpxm2', 'Matn4', 'Fndc1', 'Ahnak2', 'Apln', 'Postn', ],
                    'C1': ['Inhba', 'Rasd1', 'Prlr', 'Col23a1', 'Bmp7', 'Lef1', 'Bcl2', 'Tfap2a', 'Pappa2', 'Notum', 'Tfap2c', 'Tmem176b', 'Wnt6', 'Wnt10a', 'Bambi', 'Tmem176a', 'Ptgir', 'Slc26a7', 'Eva1a', 'Stac', 'Irf8', 'Ndnf', ] + 
    ['Slc6a6', 'Angptl4', 'Tns3', 'Gadd45a', ] +
                        ['Igfbp3', 'Bmp4', 'Fgfr1', 'Serpine2', 'Spon1', 'Runx3', 'Trib1', 'Wif1', 'Fgf7', 'Carmn', 'Rspo3', 'Dll1', 'Tuba1c', 'Gcnt2', 'Il17ra', 'Tnn', ],
                    'C2': ['Mmp11', 'Acta2', 'Cnrip1', 'Tmem140', 'Ociad2', 'Myo1e', 'Corin', ] + 
    ['Slc6a6', 'Angptl4', 'Tns3', 'Gadd45a', ] +
                        ['Mif', 'Chchd10', 'Lrrc15', 'Dkk3', 'Bok', 'Ahnak2', 'Adamts15', 'Cdc42ep3', 'Nrep', 'Arhgdib', 'Pard6g', 'Abi3bp', 'Mamdc2', 'Adam12', 'Dhrs3', ],
                    'D1': ['Chrm2', 'Fam83a', 'Hapln1', 'Hsd17b7', 'Kank1', 'Ranbp3l', 'Spata13', ] + 
    ['Dusp6', 'Emilin3', 'Itga8', 'Nrp1', 'Pcdh7', 'Scml4', ] + 
                         ['Abca9', 'Cdkn2b', 'Dennd2a', 'Mme', 'Mpdz', 'Nes', 'Sema7a', 'Slc4a4', 'Sulf1', 'Tln2', 'Tspan15', 'Vwde', 'Btg2', 'Efna5', 'Ybx3', 'C1qtnf1', 'Sdc2', 'Pcbd1', 'Apela', 'Elovl6', 'Pid1', 'Ranbp3l', ],
                    'D2': ['Chrm2', 'Egr2', 'Fam83a', 'Hapln1', 'Hsd17b7', 'Kank1', 'Spata13', 'Tbx3', 'Tmcc3', ] + 
    ['Dusp6', 'Emilin3', 'Itga8', 'Nrp1', 'Pcdh7', 'Scml4', ] + 
                         ['Abca9', 'Bhlhe41', 'Cdkn2b', 'Dennd2a', 'Mme', 'Mpdz', 'Nes', 'Sema7a', 'Slc4a4', 'Sulf1', 'Tln2', 'Tspan15', 'Vwde', 'Btg2', 'Efna5', 'Dpysl2', 'Zeb1', 'Tmem204', 'Cbfa2t3', 'Hspe1', 'Errfi1', 'Ybx3', 'Tnfsf9', 'Maff', 'Abca8a', 'Pcdh7', 'Egr2', 'Tshz3', 'Klf13', 'Hmcn1', 'Msi2', 'Gcnt4', 'Pcbd1', 'Apela', 'Elovl6', 'Pid1', 'Ranbp3l', ],
                    'D3': [] + 
    ['Dusp6', 'Emilin3', 'Itga8', 'Nrp1', 'Pcdh7', 'Scml4', ] + 
                         ['Maff', 'Postn', 'Gng11', 'F2r', ],
                    'E1': ['Bcam', 'Ccdc3', 'Dleu7', 'Lypd2', 'Moxd1', 'Mybph', 'Ntng1', 'Rgs16', 'Sfrp5', 'Slc2a1', 'Paqr6', 'Perp', 'Efnb3', 'Pcp4', 'Cldn22', ] +
    ['Chil1', 'Cldn1', 'Itga6', 'Klf5', ] + 
                         ['Cavin2', 'Cfh', 'Gsta4', 'Il33', 'Lmo7', 'Mdfic', 'Tenm2', 'Tmeff2', 'Wnt6', 'Ptch1', 'F11r', 'Akap12', 'Tubb2b', 'Wfdc1', 'Thbs4', 'Ndrg1', 'Rbp1', 'Sfrp1', 'Car11', 'Rasl11a', 'Fam78b', 'Plce1', 'Sult1a1', 'Anxa3', 'Jup', 'Vtn', ],
                    'E2': ['Il20rb', ] + 
    ['Chil1', 'Cldn1', 'Itga6', 'Klf5', ] + 
                         ['Col18a1', 'Lgals7', 'Gfra2', 'S100a4', 'Foxs1', 'Itih5', 'Ngfr', 'Amn', 'Plk2', ],
                    'F': ['Mbp', 'Plp1', 'Gatm', 'Mpz', 'Kcna1', 'Cldn1', 'Mal', 'Egfl8', 'Gjc3', 'Ncmap', 'Pllp', 'Bcas1', 'Plekhb1', 'Kcna2', 'Mlip', 'Gfra3', 'Pou3f1', 'Ptpre', 'Gas2l3', ] + 
                        ['Cldn1', 'Ebf2', 'Itga6', 'Sbspon', 'Krt19', 'Art3', 'Vwa1', 'Cldn19', 'Aatk', 'Rasal2', 'S100b', 'Slc6a13', ],
                    'G': ['Pappa', 'Tnc', ] + 
                         ['Osr2', 'Tmsb10', 'Ppp1r14a', 'Jund', 'Ccl7', 'Fam46a', 'Igf1', 'Gpm6b', 'Timp3', 'Abi3bp', 'Postn', 'Tspan11', 'Ccdc3', 'Rhoj', 'Hmcn2', 'Syt13', 'Foxd1', 'Col24a1', 'Nrp1', 'Dchs1', 'Mkl2', 'Sptlc2', 'Ppp3cb', 'Snx21', 'Lsamp', 'Deptor', 'Itga4', 'Llph', 'Scd1', 'Spon1', 'Lrig1', 'Col23a1', 'Ndnf', 'Phrf1', 'Plscr4', 'Col13a1', ],
}

dict_shook_axis  = {
    
                        'A': ['Sbsn', 'Aldh1a3', 'Dmkn', 'Smpd3', 'Efhd1', 'Chst1', 'Itgb7', 'Akr1c18', 'Krtdap', 'Lsmem1', 'Wnt10b', 'Sema3e', 'Fez1', 'Cotl1', 'Gpr1', 'Il33', 'Uchl1', 'Snhg11', 'Bmp7', 'Cxcl13', 'Adora2b', ] + 
                        ['Pi16', 'Anxa3', 'Efemp1', 'Spp4', 'Emilin2', 'Sfrp2', 'Pla1a', 'Igfbp4', 'Wnt2', 'Il1r2', 'Plpp3', 'Thbd', 'Prkg2', 'Limch1', 'Tek', 'Gas7', 'Islr2', 'Vegfd', 'Dact2', 'Ampd3', 'Pygl', 'Prmt7', 'Aif1l', 'Pdgfd', 'Iqsec1', 'Ackr2', 'Calcrl', 'Cd55b', 'Nt5e', 'Ptgs1', 'Srd5a1', 'Adgrg2', 'Cd55', 'Clip4', 'Kcnk5', 'Igsf1', 'Bmp8a', 'Duox1', 'Gap43', 'Nav3', 'E2f7', 'Ccn3', 'Asphd2', 'Eya4', 'Rab38', 'Cd200', ],
                        'B': ['Gpx3', 'Hmcn2', 'Ggt5', 'Bmper', 'Cp', 'Casp4', ] + 
                             ['Gsn', 'Cygb', 'Myoc', 'Tgfbi', 'Abca8a', 'Igfbp7', 'Smoc2', 'Cxcl14', 'Ocolce', 'Ltbp4', 'Ccn5', 'Mgp', 'Ifngr1', 'Capg', 'Cxcl12', 'Sparcl1', 'Nfib', 'Angptl1', 'Col6a6', 'Igfbp3', 'Lsamp', 'Nmb', 'Tgm2', 'Kitl', 'Myoc', ],
                        'C': [] + 
                             ['Enpp2', 'Ncam1', 'Trps1', 'Robo2', 'Fgfr2', 'Rspo1', 'Manc3', 'Myo1b', 'Tmem132c', 'Robo1', 'Nkd1', 'Mtss2', 'Tenm3', 'Cox4i2', ],
}

dict_shook = {
                    'A1': ['Ppm1n', 'Klb', ] + 
    ['Sbsn', 'Aldh1a3', 'Dmkn', 'Smpd3', 'Efhd1', 'Chst1', 'Itgb7', 'Akr1c18', 'Krtdap', 'Lsmem1', 'Wnt10b', 'Sema3e', 'Fez1', 'Cotl1', 'Gpr1', 'Il33', 'Uchl1', 'Snhg11', 'Bmp7', 'Cxcl13', 'Adora2b', ] +
                        ['Slc4a10', 'Snhg11', 'Slc7a6os', 'Gm26802', 'Gm3336', 'Edn1', 'Arl4d', 'Snx33', 'Adamts5', 'Gan', 'Tent5b', 'Slc7a6', 'Tdp2', 'Rapgef3', 'Il34', ],
                    'A2': ['Has2', 'Cxcl10', 'Fosl1', 'Mpp7', 'Dusp10', 'Rel', ] + 
    ['Sbsn', 'Aldh1a3', 'Dmkn', 'Smpd3', 'Efhd1', 'Chst1', 'Itgb7', 'Akr1c18', 'Krtdap', 'Lsmem1', 'Wnt10b', 'Sema3e', 'Fez1', 'Cotl1', 'Gpr1', 'Il33', 'Uchl1', 'Snhg11', 'Bmp7', 'Cxcl13', 'Adora2b', ] +
                        ['Ccl2', 'Ugdh', 'Gm20186', 'Gm12840', 'Nr4a3', 'Ccl7', 'Nfkb1', 'Fosl2', 'Dcbld2', 'Cxcl1', 'Mpp7', 'Tnfaip3', 'Gm47922', 'Gfpt2', 'Ptgs2', 'Has1', 'Gm47922', 'Gm14603', 'Pvr', 'Rbbp8', 'Hes1', 'Ngf', ],
                    'B1': ['Clu', 'Fmo2', 'Tgm2', 'Angpt4', 'Fgf9', 'Nkain4', 'Npy1r', 'Ecrg4', 'Actn1', 'Grem2', 'Angptl7', 'Paqr6', 'Adgrl2', 'Cobl', 'Wfdc1', 'Tns3', 'Lepr', 'Cobll1', ] + 
    ['Gpx3', 'Hmcn2', 'Ggt5', 'Bmper', 'Cp', 'Casp4', ] +                       
                          ['Sfrp2', 'Gpx3', 'Gas6', 'F3', 'Ccn5', 'Fzd1', 'Mbln1', 'Gas1', 'Tspo', 'Sfrp4', 'Igf1', 'Cdkn1c', 'Phlda3', 'Rbp1', 'Arhgdib', 'Col12a1', 'Lrp6', 'Fxyd6', 'Igfbp3', 'Ccn2', 'Fmo1', 'Cd151', 'Mpp6', 'Ptgis', 'Sema3d', 'Csrp1', 'Omd', 'Mfap4', 'Matn2', 'Akr1c14', 'Cpe', 'Sfrp2', 'Ptprd', 'Lgr4', 'Palld', 'Tnfaip8', ],
                    'B2': ['Stmn4', ] + 
    ['Gpx3', 'Hmcn2', 'Ggt5', 'Bmper', 'Cp', 'Casp4', ] + 
                          ['Fn1', 'Fbn1', 'Ugdh', 'Ptx3', 'Ackr1', 'Cd55', 'Galnt16', 'Smpd3', 'Tek', 'Has2', 'Efhd1', 'Tmem26', 'Ccn3', 'Cmah', 'Myoc', ],
                    'B3': ['Thbs1', 'Fbln7', ] + 
    ['Gpx3', 'Hmcn2', 'Ggt5', 'Bmper', 'Cp', 'Casp4', ] + 
                          ['Ccl2', 'Tnfaip6', 'Cilp', 'Has1', 'Cxcl14', 'Ccl7', ],
                    'B4': ['Cilp', 'Thbs4', 'Col8a1', 'Meox1', 'Adamtsl2', 'Crlf1', ] + 
    ['Gpx3', 'Hmcn2', 'Ggt5', 'Bmper', 'Cp', 'Casp4', ] + 
                          ['Mfap4', 'Lsamp', 'Ccn2', 'Fhl2', 'Prss12', 'Fibin', 'C1qtnf3', 'Tmem204', 'Galnt17', 'Rgs10', 'Sema5a', 'Myoc', 'F2r', 'Gm266', 'Cmklr1', 'Vav3', 'Lox', 'Fbln7', 'Fbln1', 'G0s2', 'Fhl2', 'Crislpd2', 'Spry1', ],
                    'B5': ['Agt', ] + 
    ['Gpx3', 'Hmcn2', 'Ggt5', 'Bmper', 'Cp', 'Casp4', ] + 
                          ['C4b', 'Ecm1', 'Abi3bp', 'Steap4', 'Vit', 'Inmt', 'Egfr', 'Nmb', 'Sned1', 'Chrdl1', 'Fmo2', 'Meox2', 'Ralgps2', 'P2ry1', 'Cpe', 'Cxcl16', 'Sult1e1', 'Adamtsl3', 'Avpr1a', 'Rbfox3', 'Vtn', 'Tnfsf13b', 'Gabra3', 'Tent5c', ],
                    'B6': ['C6', 'Aoc3', 'Phospho1', 'Fabp4', 'Olfm2', ] + 
    ['Gpx3', 'Hmcn2', 'Ggt5', 'Bmper', 'Cp', 'Casp4', ] + 
                          ['Cxcl12', 'Postn', 'Mmp14', 'Cthrc1', 'Pltp', 'Ccl19', 'Agt', 'Col14a1', 'Mlana', 'Tnmd', 'Cyp2f2', 'Cd276', 'Cyp1b1', 'C1qtnf3', 'Sema3b', 'Creb3l3', 'Fzd4', 'Mex3b', 'B4galnt1', 'Sox4', 'Prkcb', 'Peg3', 'Scg3', 'Pparg', 'Ddah1', 'Apoc1', ],
                    'C1': ['Cpz', 'Cdh4', 'Cgref1', 'Aldh3a1', 'Sectm1a', 'Gal', 'Csf1r', 'Camk2a', 'Fgfr4', 'Aldh1l2', 'Ddah1', 'Ppp2r2c', 'Slc27a1', 'Serpina3c', 'Prex1', 'Cib3', 'Adcy1', 'Tmem150c', 'Rassf4', 'Gpha2', 'Wfdc12', 'Colq', 'Miat', 'Slc27a6', ] + 
                          ['Ndufa4l2', 'Col1a2', 'Sparc', 'Col1a1', 'Cldn10', 'Cyp2f2', 'C1qtnf3', 'Ctla2a', 'Mmp3', 'Creb3l3', 'Mmp27', 'Chpf', 'Pla2g5', 'Emid1', 'Abcc4', 'Tent5c', 'Ralgps2', 'Prkcb', 'Ltbp2', 'Serpina3m', 'Bcat1', 'Pla2g2e', 'Pycr1', 'Ccbe1', 'Nat8f3', 'Arhgap15', 'Slc16a3', ],
                    'C2': ['Lrrc15', 'Col11a1', 'Aqp1', 'Cd200', 'Stmn2', 'Bbc3', 'Nrtn', ] + 
                          ['Lmna', 'Postn', 'Tpm2', 'Robo2', 'Ncam1', 'Stmn1', 'Wif1', 'Bcl11b', 'Col7a1', 'Fscn1', 'Mylk', 'Pianp', 'Rasgrp2', 'Ifitm1', 'Nrp2', 'Tagln', 'Igfbp2', 'Foxd2os', 'Itga8', 'Marcksl1', 'Adamts18', 'a', ],
                    'C3': ['Mmp11', 'Plxdc1', 'Mamdc2', 'Ociad2', 'Rab27a', 'Chchd10' ] + 
                          ['Lrrc15', 'Mdk', 'Chchd10', 'Mafb', 'Adam12', 'F2r', 'Sox4', 'Dhrs3', 'Gadd45a', 'Bok', 'Pawr', 'Mgp', 'Aqp1', 'Ednrb', 'Ctsc', 'Cipc', 'Corin', 'Cdk15', 'Mlana', 'Nme4', 'Haus4', 'Myo1e', ],
                    'C4': ['Chst8', 'Gucy1b1', 'Wnt6', 'Bcl2', 'Notum', 'Wnt10a', 'Igf2', 'Mapk13', ] + 
                          ['Prlr', 'Rasd1', 'Slc26a7', 'a', 'Crabp1', 'Igfbp3', 'Cacna1g', 'Pappa2', 'Rspo1', 'Ndnf', 'Myo1b', 'Snai1', 'Wif1', 'Skil', 'Ptger3', 'Lef1', 'Spock1', 'Rb1', 'Pgf', 'Dkk2', 'Ackr4', 'Cpne5', 'Apcdd1', 'Slc6a6', 'Tns3', 'Tmem132c', 'Smim3', 'Sms', 'Sik1', ],
                    'C5': ['Lrrn1', 'Msantd1', 'Spock1', 'Sox18', 'Alcam', 'Aard', 'Galnt9', 'Sdk2', 'Rgs6', ] + 
                          ['Mdk', 'Ptch1', 'Cd24a', 'Lamc3', 'Wnt5a', 'Serpine2', 'Edn3', 'Ss18', 'Alpl', 'Prlr', 'Slc16a2', 'Gldn', 'Zfand5', 'Mpped2', 'Hmgn2', 'Frem1', 'Id1', 'Ncald', 'Cacna1g', 'H1fx', 'Cyp51', 'Aplp1', 'Pappa2', 'Fdps', 'Hey2', 'Epha7', 'Sdc3', 'Tpd52l1', 'Megf6', 'Draxin', 'Acot1', 'Ephb6', 'Fabp5', 'Ndp', 'Prdm1', 'Tfap2c', 'Snhg11', ],
                    'C6': ['Nrg2', 'Corin', 'Sostdc1', 'Chodl', 'Ptprz1', 'Ints6l', 'Nav2', 'Crym', 'Cntn1', 'Hal', 'Stac', 'Rspo4', 'Cmtm8', ] + 
                          ['Tmem176b', 'Tmem176a', 'Wif1', 'Scube2', 'Crabp1', 'Edn3', 'Ptger3', 'Masp1', 'Inhba', 'Alx4', 'Mir155hg', 'Trpm3', 'Zcchc18', 'Kctd12', 'Itga9', 'Net1', 'Rgs2', 'Dusp10', 'Matn4', 'Notum', ],
                    'C7': [] + 
                          ['Ccl19', 'Ndufa4l2', 'Mfap4', 'Eln', 'Tnmd', 'Cyp2f2', 'Tsc22d3', 'Rtn4r', 'Ltbp2', 'Gpm6b', 'Fam180a', 'Bcl11l', 'Lsamp', 'Pid1', 'Lgr5', 'Sectm1a', 'Prkcb', 'Entpd1', 'Gm12407', 'Seprina3c', 'Myo1b', 'H2-Q7', 'Bmp4', 'Cystm1', 'Cyp1b1', 'Wnt5b', 'Id1', 'Syt13', 'Etv1', 'Eya1', ],
                    'C8': [] + 
                          ['Ctsk', 'Mmp14', 'Mlana', 'Mfap2', 'Ctla2a', 'Ctsh', 'C1qtnf3', 'Adam12', 'Mdk', '3300005D01Rik', 'Zfp469', 'Cyp1b1', 'Creb3l3', 'Cyp7b1', 'Gng13', 'Cd276', 'Sowahc', 'Prkcb', 'Cdh4', 'Mex3b', 'Sox4', 'Marcksl1', ],

}

In [ ]:
sc.tl.leiden(abassi_2020_ctrl_fb, resolution=10, key_added='leiden_many')
sc.pl.umap(abassi_2020_ctrl_fb, color='leiden_many', legend_loc='on data')

In [ ]:
sc.tl.leiden(buechler_2021_ctrl_fb, resolution=10, key_added='leiden_many')
sc.pl.umap(buechler_2021_ctrl_fb, color='leiden_many', legend_loc='on data')

In [ ]:
sc.tl.leiden(haensel_2021_ctrl_wounding_fb, resolution=10, key_added='leiden_many')
sc.pl.umap(haensel_2021_ctrl_wounding_fb, color='leiden_many', legend_loc='on data')

In [ ]:
sc.tl.leiden(phan_2020_ctrl_21d_fb, resolution=10, key_added='leiden_many')
sc.pl.umap(phan_2020_ctrl_21d_fb, color='leiden_many', legend_loc='on data')

In [ ]:
sc.tl.leiden(shook_2020_ctrl_fb, resolution=10, key_added='leiden_many')
sc.pl.umap(shook_2020_ctrl_fb, color='leiden_many', legend_loc='on data')

## Overlap between datasets

In [ ]:
diff, legend_loc = 0.003, 'right margin'
list_letters = ['a', 'b', 'h', 'p', 's']
dict_adatas = {'a': abassi_2020_ctrl_fb, 'b': buechler_2021_ctrl_fb, 'h': haensel_2021_ctrl_wounding_fb, 'p': phan_2020_ctrl_21d_fb, 's':shook_2020_ctrl_fb}
dict_cats = {'a': dict_abassi, 'b': dict_buechler, 'h': dict_haensel, 'p': dict_phan, 's':dict_shook}
dict_keys = {'a': 'cluster_abassi', 'b': 'cluster_buechler', 'h': 'cluster_haensel', 'p': 'cluster_phan', 's': 'cluster_shook'}

### Overlap Abassi VS rest

In [ ]:
letter_max = 'a'

for letter in list_letters:
    assign_cats(dict_adatas[letter], dict_cats=dict_cats[letter_max], min_score=0.55, quantile_gene_sel=0.95, 
            diff=diff, key_added=dict_keys[letter_max], column_groupby='leiden_many', others_name='U')
    dict_adatas[letter].obs[dict_keys[letter_max]] = dict_adatas[letter].obs[dict_keys[letter_max]] + letter_max
    sc.pl.umap(dict_adatas[letter], color=['leiden_many', dict_keys[letter_max]], legend_loc=legend_loc)

### Overlap Buechler VS rest

In [ ]:
letter_max = 'b'

for letter in list_letters:
    assign_cats(dict_adatas[letter], dict_cats=dict_cats[letter_max], min_score=0.55, quantile_gene_sel=0.95, 
            diff=diff, key_added=dict_keys[letter_max], column_groupby='leiden_many', others_name='U')
    dict_adatas[letter].obs[dict_keys[letter_max]] = dict_adatas[letter].obs[dict_keys[letter_max]] + letter_max
    sc.pl.umap(dict_adatas[letter], color=['leiden_many', dict_keys[letter_max]], legend_loc=legend_loc)

### Overlap Haensel VS rest

In [ ]:
letter_max = 'h'

for letter in list_letters:
    assign_cats(dict_adatas[letter], dict_cats=dict_cats[letter_max], min_score=0.55, quantile_gene_sel=0.95, 
            diff=diff, key_added=dict_keys[letter_max], column_groupby='leiden_many', others_name='U')
    dict_adatas[letter].obs[dict_keys[letter_max]] = dict_adatas[letter].obs[dict_keys[letter_max]] + letter_max
    sc.pl.umap(dict_adatas[letter], color=['leiden_many', dict_keys[letter_max]], legend_loc=legend_loc)

### Overlap Phan VS rest

In [ ]:
letter_max = 'p'

for letter in list_letters:
    assign_cats(dict_adatas[letter], dict_cats=dict_cats[letter_max], min_score=0.55, quantile_gene_sel=0.95, 
            diff=diff, key_added=dict_keys[letter_max], column_groupby='leiden_many', others_name='U')
    dict_adatas[letter].obs[dict_keys[letter_max]] = dict_adatas[letter].obs[dict_keys[letter_max]] + letter_max
    sc.pl.umap(dict_adatas[letter], color=['leiden_many', dict_keys[letter_max]], legend_loc=legend_loc)

### Overlap Shook VS rest

In [ ]:
letter_max = 's'

for letter in list_letters:
    assign_cats(dict_adatas[letter], dict_cats=dict_cats[letter_max], min_score=0.55, quantile_gene_sel=0.95, 
            diff=diff, key_added=dict_keys[letter_max], column_groupby='leiden_many', others_name='U')
    dict_adatas[letter].obs[dict_keys[letter_max]] = dict_adatas[letter].obs[dict_keys[letter_max]] + letter_max
    sc.pl.umap(dict_adatas[letter], color=['leiden_many', dict_keys[letter_max]], legend_loc=legend_loc)

## PAGA of clusters

In [ ]:
adata, cluster_name = abassi_2020_ctrl_fb, 'cluster_abassi'
sc.tl.paga(adata, groups=cluster_name)

fig, axs = plt.subplots(1, 3, figsize=(22, 5))
sc.pl.umap(adata, color=cluster_name, legend_loc='on data', ncols=5, ax=axs[0], show=False, frameon=False, title='', legend_fontsize=10)
sc.pl.paga(adata, ax=axs[1], frameon=False, show=False,)
sc.pl.paga(adata, ax=axs[2], frameon=False, solid_edges='connectivities_tree')

In [ ]:
adata, cluster_name = buechler_2021_ctrl_fb, 'cluster_buechler'
sc.tl.paga(adata, groups=cluster_name)

fig, axs = plt.subplots(1, 3, figsize=(22, 5))
sc.pl.umap(adata, color=cluster_name, legend_loc='on data', ncols=5, ax=axs[0], show=False, frameon=False, title='', legend_fontsize=10)
sc.pl.paga(adata, ax=axs[1], frameon=False, show=False,)
sc.pl.paga(adata, ax=axs[2], frameon=False, solid_edges='connectivities_tree')

In [ ]:
adata, cluster_name = haensel_2021_ctrl_wounding_fb, 'cluster_haensel'
sc.tl.paga(adata, groups=cluster_name)

fig, axs = plt.subplots(1, 3, figsize=(22, 5))
sc.pl.umap(adata, color=cluster_name, legend_loc='on data', ncols=5, ax=axs[0], show=False, frameon=False, title='', legend_fontsize=10)
sc.pl.paga(adata, ax=axs[1], frameon=False, show=False,)
sc.pl.paga(adata, ax=axs[2], frameon=False, solid_edges='connectivities_tree')

In [ ]:
adata, cluster_name = phan_2020_ctrl_21d_fb, 'cluster_phan'
sc.tl.paga(adata, groups=cluster_name)

fig, axs = plt.subplots(1, 3, figsize=(22, 5))
sc.pl.umap(adata, color=cluster_name, legend_loc='on data', ncols=5, ax=axs[0], show=False, frameon=False, title='', legend_fontsize=10)
sc.pl.paga(adata, ax=axs[1], frameon=False, show=False,)
sc.pl.paga(adata, ax=axs[2], frameon=False, solid_edges='connectivities_tree')

In [ ]:
adata, cluster_name = shook_2020_ctrl_fb, 'cluster_shook'
sc.tl.paga(adata, groups=cluster_name)

fig, axs = plt.subplots(1, 3, figsize=(22, 5))
sc.pl.umap(adata, color=cluster_name, legend_loc='on data', ncols=5, ax=axs[0], show=False, frameon=False, title='', legend_fontsize=10)
sc.pl.paga(adata, ax=axs[1], frameon=False, show=False,)
sc.pl.paga(adata, ax=axs[2], frameon=False, solid_edges='connectivities_tree')

## Relationships between datasets

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20, 15))
for row_idx, adata in enumerate([abassi_2020_ctrl_fb, buechler_2021_ctrl_fb, haensel_2021_ctrl_wounding_fb, phan_2020_ctrl_21d_fb, shook_2020_ctrl_fb]):
    for col_idx, cluster_list in enumerate(['cluster_abassi', 'cluster_buechler', 'cluster_haensel', 'cluster_phan', 'cluster_shook']):
        sc.pl.umap(adata, color=cluster_list, legend_loc='on data', ncols=5, ax=axs[row_idx, col_idx], show=False, frameon=False, title='', legend_fontsize=10)
plt.axis('off')
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20, 15))
for row_idx, adata in enumerate([abassi_2020_ctrl_fb, buechler_2021_ctrl_fb, haensel_2021_ctrl_wounding_fb, phan_2020_ctrl_21d_fb, shook_2020_ctrl_fb]):
    for col_idx, cluster_list in enumerate(['cluster_abassi', 'cluster_buechler', 'cluster_haensel', 'cluster_phan', 'cluster_shook']):
        sc.pl.umap(adata, color=cluster_list, ncols=5, ax=axs[row_idx, col_idx], show=False, frameon=False, title='', legend_fontsize=10)
plt.axis('off')
plt.tight_layout()

### Matrix of relationships
This matrix will help find relationships between datasets. The matrices shown in the final form are two: the reciprocal matrix and the relationship matrix. The reciprocal matrix is the one where there is a full pairing between two clusters from two datasets. For instance, cluster A1a in dataset A and A5b in dataset B appear in the matrix if cluster A1a appears in the location of A5b in dataset B, AND A5b appears in the location of A1a in dataset A. On the other hand, a match on the relationship matrix appears when if cluster A1a appears in the location of A5b in dataset B, OR A5b appears in the location of A1a in dataset A. This means that there might be a slight relationship between one of the clusters, but it might not that strong. 

Also, important, bigger clusters (that encompass 2 or more clusters in the other dataset) might appear in the matrices as one row of matches.

In [ ]:
dict_adatas = {'a': abassi_2020_ctrl_fb, 'b': buechler_2021_ctrl_fb, 'h': haensel_2021_ctrl_wounding_fb, 'p': phan_2020_ctrl_21d_fb, 's':shook_2020_ctrl_fb}
dict_cats = {'a': dict_abassi, 'b': dict_buechler, 'h': dict_haensel, 'p': dict_phan, 's':dict_shook}
dict_keys = {'a': 'cluster_abassi', 'b': 'cluster_buechler', 'h': 'cluster_haensel', 'p': 'cluster_phan', 's': 'cluster_shook'}

In [ ]:
def get_graphs_ab(a, b):
    adata_a, adata_b = dict_adatas[a], dict_adatas[b]
    cluster_name_a, cluster_name_b = dict_keys[a], dict_keys[b]
    cats_a, cats_b = adata_a.obs[cluster_name_a].cat.categories, adata_b.obs[cluster_name_b].cat.categories

    df_a = pd.DataFrame(0, index=cats_a, columns=cats_b)
    df_b = pd.DataFrame(0, index=cats_a, columns=cats_b)
    for cat_a in cats_a:
        for cat_b in cats_b:
            try:
                labels_a, labels_b = adata_a.obs[cluster_name_a], adata_a.obs[cluster_name_b]
                df_a.loc[cat_a, cat_b] = ((labels_a == cat_a) & (labels_b == cat_b)).sum() / (labels_a == cat_a).sum()
            except:
                pass
            try:
                labels_a, labels_b = adata_b.obs[cluster_name_a], adata_b.obs[cluster_name_b]
                df_b.loc[cat_a, cat_b] = ((labels_a == cat_a) & (labels_b == cat_b)).sum() / (labels_b == cat_b).sum()
            except:
                pass
    
    return df_a, df_b

In [ ]:
all_labels = []
for letter in list_letters:
    all_labels += list(dict_adatas[letter].obs[dict_keys[letter]].cat.categories)
    
megadf_reciprocal, megadf_complementary = pd.DataFrame(0, index=all_labels, columns=all_labels), pd.DataFrame(0, index=all_labels, columns=all_labels)


for a in list_letters:
    for b in list_letters:
        if a != b:
            df_a, df_b = get_graphs_ab(a, b)
            megadf_reciprocal.loc[df_a.index, df_a.columns] = np.sqrt(df_a * df_b)
            megadf_complementary.loc[df_a.index, df_a.columns] = (df_a + df_b) / 2

In [ ]:
sns.clustermap(megadf_reciprocal, xticklabels=True, yticklabels=True)

In [ ]:
sns.clustermap(megadf_complementary, xticklabels=True, yticklabels=True)

From the analysis of populations we obtained a graph of fibroblast populations from 5 different articles. 

![alt text](images/mouse_fb_mapping/mouse_fb_graph_complete_v0.png)

Then we merge the populations into the "minimum" ones. These might be subject to changes.

![alt text](images/mouse_fb_mapping/mouse_fb_graph_merge_v0.png)


## Rearranging populations

We are going to create a dictionary of merged populations.

In [ ]:
dict_cats_clusters = {'a1*': ['Abl2', 'Ccl2', 'Ccl7', 'Cxcl1', 'Cxcl10', 'Dusp10', 'Fosl1', 'Gfpt2', 'Gm12840', 'Has1', 'Has2', 'Hk2', 'Hmox1', 'Nfkb1', 'Ptx3', 'Ugdh'], # 17
                      'a2*': ['Ak1c18', 'Cotl1', 'Efhd1', 'Fez1', 'Il18', 'Lims2', 'Mustn1', 'Pts', 'Adora2b', 'Aldh1a3', 'Chst1', 'Gan', 'Gap43', 'Pamr1', 'Snhg11'],
                      'a3*': ['Cdca3', 'Cdca8', 'Stmn1', 'Cks2', 'Birc5', 'Lockd', 'Top2a', 'Ccna2', 'Ube2c', 'Diaph3', 'Spc25', 'Tpx2', 'Mki67', 'Pbk', 'Prc1', 
                              'Cdk1', 'Hist1h2ap', 'Tacc3', 'Racgap1', 'Mxd3',], # 20
                      'a/b*': ['Ackr1', 'Ccn3', 'Cmah', 'Efhd1', 'Fbn1', 'Fn1', 'Galnt16', 'Has2', 'Smpd3', 'Stmn4', 'Tmem26', ],
                      'b1*': ['Adam12', 'Ccl19', 'Cd36', 'Col6a6', 'Creb3l3', 'Cthrc1', 'Cxcl12', 'Hmcn2', 'Nrep', 'Pltp', 'Adam19', 'Adam23', 'Agt', 'Aoc3', 'C1qtnf3', 
                             'Cdkn1c', 'Col14a1', 'Col4a1', 'Cpe', 'Cygb', 'Cyp1b1', 'Cyp2f2', 'Fabp4', 'Fbln7', 'Gdf15', 'Ggt5', 'Gpc6', 'Il1r2', 'Itih5', 'Mex3b', 'Olfm2', 
                             'Peg3', 'Rassf2', 'Sema3b', 'Sox4', 'Tnmd', 'Tspan7', 'Zfp385a', ],
                      'b2*': ['Cadm2', 'Angptl4', 'Galnt5', 'Mex3b'], # 4
                      'b3*': ['Adamtsl2', 'Bmp5', 'Cilp', 'Col8a1', 'Fbln7', 'Fhl2', 'Fibin', 'G0s2', 'Meox1', 'Mest', 'Prss12', 'Thbs4', 'Cmklr1', 'Col6a6', 'Crlf1', 'Dlk1', 
                              'F2r', 'Fap', 'Gm266', 'Hmcn2', ],  # 20,
                      'b4*': ['Abcc9', 'Akr1c14', 'Cilp', 'Col12a1', 'Crispld2', 'Eln', 'Fibin', 'Igf1', 'Lrp6', 'Mfap4', 'Tgm2', 'Actn1', 'Adamtsl2', 
                              'Anxa3', 'Arhgdib', 'Art3', 'Cpe', 'Ctla2a', ], # 18
                      'b5*': ['C2', 'Cfh', 'Chrdl1', 'Cp', 'Cxcl16', 'Naalad2', 'Scn7a', 'Sned1', 'Steap4', 'Vit', 'Vtn', 'Abi3bp', 'Ace', 'Agt', 'C1ra', 'C4b', 'Cfb', 
                              'Cxcl12', 'Ecm1', 'F3', 'Fmo2', 'Gabra3', 'Meox2', 'Nmb', 'Plac8', 'Ube2l6', ], #26
                      'b6*': ['Acot1', 'Adamtsl3', 'Apod', 'Ccl11', 'Clec1a', 'Col26a1', 'Col8a1', 'Cp', 'Cpe', 'Gfra1', 'Gpc3', 'Matn2', 'Myoc', 'Ngfr', 'Nipal1', 
                              'P2ry14', 'Pdrg1', 'Plxdc1', 'Rasgrp2', 'Reln', 'Sdc3', 'Smoc2', 'Sox8', 'Steap4', 'Trf', 'Vit', 'Vtn', 'Vwa1', ], 
                      'b/c*': ['C1qtnf3', 'Mkx', 'Tnmd', 'Adam12', 'Akr1cl', 'Col12a1', 'Creb3l3', 'Crp', 'Ctla2a', 'Ctsh', 'Cyp1b1', 
                              'Gfra2', 'Gng13', 'Hmcn1', 'Iqgap2', 'Nkain4', 'Pltp', 'Prkcb', 'Ptgfr', 'Rtn4r', ],
                      'c1*': ['Abcc4', 'Cgref1', 'Creb3l3', 'Fcgr3', 'Wisp2', 'Adcy1', 'Ahrr', 'Aldh1l2', 'Angptl1', 'Arhgef19', 'Bcat1', 'Bcl11b', 'Bdh2', 'Cdh4', 'Cib3', 'Cldn10', 
                              'Cyp4b1', 'Gpm6a', 'Ltbp2', 'Mmp27', 'Penk', 'Pi15', 'Prkcb', 'Tmem97', ],
                      'c2*': ['Ccdc42', 'Col13a1', 'Col7a1', 'Fxyd6', 'Grem1', 'Igfbp2', 'Sema3a', 'Stc1', 'Wnt5a', 'Ackr4', 'Ahrr', 'Cdh13', 'F13a1', 'Hmcn1', 'Mamdc2', 'Nov', 
                              'Ptchd4', 'Rgcc', 'Rspo1', 'Scg3', 'Thbs2', 'Twist2', 'Wisp1', ],
                      'c3*': ['Ccl19', 'Ccl2', 'Ccl7', 'Ccl8', 'Cpxm1', 'Ets2', 'Ltbp1', 'Mmp3', 'Sowahc', 'Stk40', 'Vcam1', ],
                      'd1*': ['Tspan15', 'Tbx3', 'Spry1', 'Shc4', 'Pcbd1', 'Nrp1', 'Nes', 'Meox2', 'Hsd17b7', 'Hapln1', 'Foxd1', 'Fam83a', 'F2r', 'Efna5', 'Dennd2a', 'Chrm2', 
                              'C1qtnf1', 'Aqp1', 'Apela', 'Amn', 'Abca9', 'Abca8a', ],
                      'd2*': ['Cd200', 'Col11a1', 'Mafb', 'Robo2', 'Tagln', 'Tnmd', 'Actg2', 'Actn1', 'Aqp1', 'Bok', 'Cck', 'Col7a1', 'Cox4i2', 'Ednrb', 'F2r', 'Ifitm1', 'Itga11', 'Itga8', 'Kif26b', 
                              'Lrrc15', 'Megf6', 'Mylk', 'Nradd', 'Omd', 'Pfn2', 'Pianp', 'Postn', 'Slc6a6', 'Stmn1', 'Stmn2', 'Wif1', 'a', ],
                      'd3*': ['Auts2', 'Bmp4', 'Bmp7', 'Cd24a', 'Crabp1', 'Enpp2', 'Eva1a', 'Gldn', 'Inhba', 'Lef1', 'Ndnf', 'Notum', 'Pappa2', 'Ptk7', 'Rspo3', 'Serpine2', 'Slc16a2', 'Spon1', 
                              'Tmem176a', 'Tmem176b', 'Trps1', 'Wif1', 'a', ],
                      'd4*': ['Abi3bp', 'Acta2', 'Adam12', 'Adamts15', 'Arhgdib', 'Bok', 'Cdc42ep3', 'Chchd10', 'Cnrip1', 'Corin', 'Dhrs3', 'Dkk3', 'Ednrb', 'F2r', 'Haus4', 'Lrrc15', 
                              'Mafb', 'Mamdc2', 'Mdk', 'Mgp', 'Mif', 'Mmp11', 'Nme4', 'Nrep', 'Ociad2', 'Plxdc1', 'Rab27a', 'Tmem140', ],
                      'd5*': ['Cilp', 'Cmklr1', 'Cpxm2', 'Eln', 'Fam180a', 'Il15', 'Mmp16', 'Nrep', 'P2ry1', 'Pi16', 'Rnf112', 'Tspan18', 'Ackr4', 'Ccl19', 'Col6a6', 'Cyp1b1', 
                              'Cyp2f2', 'Cystm1', 'Dkk2', 'Ednra', 'Fam69a', 'Gas1', 'Hmcn1', ]}

In [ ]:
list_unfiltered_a1 = sorted(set(dict_abassi['A3']) | set(dict_shook['A2']))
print(list_unfiltered_a1)

In [ ]:
list_unfiltered_a2 = sorted(set(dict_abassi['A2']) | set(dict_shook['A2']))
print(list_unfiltered_a2)

In [ ]:
list_unfiltered_a3 = sorted(set(dict_abassi['A1']))
print(list_unfiltered_a3)

In [ ]:
list_unfiltered_ab = sorted(set(dict_shook['B2']))
print(list_unfiltered_ab)

In [ ]:
list_unfiltered_b1 = sorted(set(dict_abassi['B6']) | set(dict_shook['B6']) | set(dict_haensel['B4']))
print(list_unfiltered_b1)

In [ ]:
list_unfiltered_b2 = sorted(set(dict_buechler['B4']) | set(dict_haensel['B3']))
print(list_unfiltered_b2)

In [ ]:
list_unfiltered_b3 = sorted(set(dict_abassi['B5']) | set(dict_haensel['B7']) | set(dict_shook['B4']))
print(list_unfiltered_b3)

In [ ]:
list_unfiltered_b4 = sorted(set(dict_abassi['B4']) | set(dict_haensel['B2']) | set(dict_shook['B1']))
print(list_unfiltered_b4)

In [ ]:
list_unfiltered_b5 = sorted(set(dict_abassi['B2']) | set(dict_abassi['B3']) | set(dict_buechler['A3']) | set(dict_haensel['B8']) | set(dict_shook['B5']))
print(list_unfiltered_b5)

In [ ]:
list_unfiltered_b6 = sorted(set(dict_abassi['B1']) | set(dict_haensel['B9']) | set(dict_phan['A5']))
print(list_unfiltered_b6)

In [ ]:
list_unfiltered_bc = sorted(set(dict_buechler['C2']) | set(dict_shook['C8']))
print(list_unfiltered_bc)

In [ ]:
list_unfiltered_c1 = sorted(set(dict_abassi['C1']) | set(dict_buechler['C2']) | set(dict_shook['C8']))
print(list_unfiltered_c1)

In [ ]:
list_unfiltered_c2 = sorted(set(dict_abassi['C2']) | set(dict_haensel['A6']) | set(dict_phan['A1']))
print(list_unfiltered_c2)

In [ ]:
list_unfiltered_c3 = sorted(set(dict_abassi['C3']) | set(dict_haensel['A2']) | set(dict_phan['A2']))
print(list_unfiltered_c3)

In [ ]:
list_unfiltered_c4 = sorted(set(dict_haensel['A4']) | set(dict_haensel['A5']) | set(dict_phan['A7']))
print(list_unfiltered_c4)

In [ ]:
list_unfiltered_d1 = sorted(set(dict_haensel['D1']) | set(dict_phan['D3']) | set(dict_phan['D1']) | set(dict_phan['D2']))
print(list_unfiltered_d1)

In [ ]:
list_unfiltered_d1 = sorted(set(dict_haensel['D1']) | set(dict_phan['D3']) | set(dict_phan['D1']) | set(dict_phan['D2']))
print(list_unfiltered_d1)

In [ ]:
list_unfiltered_d2 = sorted(set(dict_abassi['D1']) | set(dict_buechler['D1']) | set(dict_phan['B1']) | set(dict_phan['B2']) | set(dict_phan['B3']) | set(dict_shook['C2']))
print(list_unfiltered_d2)

In [ ]:
list_unfiltered_d3 = sorted(set(dict_abassi['D2']) | set(dict_buechler['D2']) | set(dict_phan['C1']) | set(dict_shook['C4']) | set(dict_shook['C5']) | set(dict_shook['C6']))
print(list_unfiltered_d3)

In [ ]:
list_unfiltered_d4 = sorted(set(dict_phan['C2']) | set(dict_shook['C3']))
print(list_unfiltered_d4)

In [ ]:
list_unfiltered_d5 = sorted(set(dict_haensel['D4']) | set(dict_haensel['D5']) | set(dict_phan['A3']) | set(dict_shook['C7']))
print(list_unfiltered_d5)

In [ ]:
list_unfiltered = list_unfiltered_d5

In [ ]:
xoxo = abassi_2020_ctrl_fb
sc.pl.umap(xoxo, color=[i for i in list_unfiltered if i in xoxo.var_names],
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
xoxo = buechler_2021_ctrl_fb
sc.pl.umap(xoxo, color= [i for i in list_unfiltered if i in xoxo.var_names],
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
xoxo = haensel_2021_ctrl_wounding_fb
sc.pl.umap(xoxo, color= [i for i in list_unfiltered if i in xoxo.var_names],
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
xoxo = phan_2020_ctrl_21d_fb
sc.pl.umap(xoxo, color= [i for i in list_unfiltered if i in xoxo.var_names],
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
xoxo = shook_2020_ctrl_fb
sc.pl.umap(xoxo, color= [i for i in list_unfiltered if i in xoxo.var_names],
           legend_loc='on data', cmap=magma, use_raw=False, ncols=3)

In [ ]:
for i in list_unfiltered_d5:
    print(i)